### good_data_점검하기

In [ ]:
from pathlib import Path

# === 설정 ===
base_dir = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250930/20250930_good_data")

# === 파일 목록 수집 ===
jpg_files = {p.stem for p in base_dir.glob("*.jpg")}
txt_files = {p.stem for p in base_dir.glob("*.txt")}

# === 매칭되지 않는 파일 찾기 ===
jpg_only = sorted(jpg_files - txt_files)
txt_only = sorted(txt_files - jpg_files)

# === 출력 ===
print(f"📸 JPG만 있고 TXT 없는 파일 ({len(jpg_only)}개):")
for name in jpg_only:
    print(f" - {name}.jpg")

print("\n📝 TXT만 있고 JPG 없는 파일 ({len(txt_only)}개):")
for name in txt_only:
    print(f" - {name}.txt")


### 객체확인

In [ ]:
from pathlib import Path
from collections import Counter

# ====== 경로 설정 ======
LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250721/20250721_good_data")
#LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250725/20250725_good_data")
#LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data")
#LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250930/20250930_good_data")

# YOLO 클래스 이름 매핑
classes = [
    "Divot",         # 0
    "Fixed_Divot",   # 1
    "Diseased_Grass",# 2
    "Confused_Object",# 3
    "Pole",          # 4
    "Sprinkler",     # 5
    "Drain",         # 6
    "Golf ball"      # 7
]

counter = Counter()

for txt_file in LABEL_DIR.glob("*.txt"):
    if txt_file.name == "classes.txt":     # ★ classes.txt는 건너뛰기
        continue
    with open(txt_file, "r") as f:
        for line in f:
            parts = line.strip().split()
            if parts:
                cls_id = int(parts[0])
                counter[cls_id] += 1

# ====== 결과 출력 ======
print("=== 클래스별 객체 개수 ===")
for i, cls_name in enumerate(classes):
    print(f"{cls_name:<15}: {counter.get(i, 0)}")

print("\n총 객체 수:", sum(counter.values()))

### 데이터복사

In [1]:
from pathlib import Path
import shutil

# ===== 경로 설정 =====
DST = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251013_merge_data")
DST.mkdir(parents=True, exist_ok=True)  # 목적지 폴더 생성

SRC_LIST = [
    Path("/home/dw/ws_job_msislab/Golf_Project/data/20250721/20250721_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/20250725/20250725_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/20250930/20250930_good_data"),
]

# ===== 복사 실행 =====
for src in SRC_LIST:
    target = DST / src.name  # 예: .../20250914_merge_data/20250721_good_data
    print(f"복사 중: {src}  →  {target}")
    # dirs_exist_ok=True 로 이미 존재해도 덮어씀
    shutil.copytree(src, target, dirs_exist_ok=True)

print(f"\n[완료] 총 {len(SRC_LIST)}개 폴더를 {DST} 아래로 복사했습니다.")

복사 중: /home/dw/ws_job_msislab/Golf_Project/data/20250721/20250721_good_data  →  /home/dw/ws_job_msislab/Golf_Project/data/for_study/20251013_merge_data/20250721_good_data
복사 중: /home/dw/ws_job_msislab/Golf_Project/data/20250725/20250725_good_data  →  /home/dw/ws_job_msislab/Golf_Project/data/for_study/20251013_merge_data/20250725_good_data
복사 중: /home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data  →  /home/dw/ws_job_msislab/Golf_Project/data/for_study/20251013_merge_data/20250904_good_data
복사 중: /home/dw/ws_job_msislab/Golf_Project/data/20250930/20250930_good_data  →  /home/dw/ws_job_msislab/Golf_Project/data/for_study/20251013_merge_data/20250930_good_data

[완료] 총 4개 폴더를 /home/dw/ws_job_msislab/Golf_Project/data/for_study/20251013_merge_data 아래로 복사했습니다.


### 데이터 분할 _ 랜덤시드

In [ ]:
# split_inplace_balanced_random.py
# - 대상 폴더(각 good_data) 내부 파일을 '이동'하여 images/{train,val,test}, labels/{train,val,test}로 재배치
# - 파일명 규칙/시퀀스 무시: 전역+블록 셔플로 강하게 섞은 뒤 확률적(노이즈) 층화로 클래스 비율 유지
# - data.yaml 생성 없음
# - 파괴적 작업(이동)입니다. 필요하면 실행 전 백업하세요.

from pathlib import Path
from collections import Counter
import random, shutil

# ========== 작업 대상 폴더 ==========
TARGETS = [
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250922_merge_data/20250721_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250922_merge_data/20250725_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250922_merge_data/20250904_good_data"),
]

# ========== 분할 비율 / 하이퍼파라미터 ==========
RATIOS = {"train": 0.64, "val": 0.26, "test": 0.10}  # 합=1.0
SIZE_WEIGHT = 0.05   # 샘플수 목표 오차 가중치
JITTER = 0.20        # 비용에 더하는 무작위 노이즈(↑ 더 랜덤)
BLOCK_SIZE = 16      # 블록 셔플 크기(↑ 더 섞임)
LABEL_TOL = 0.05     # split별 라벨 타깃 과충족 허용률(5% 여유)
SEED = 38
random.seed(SEED)

# ========== 데이터 스펙 ==========
IMG_EXTS = [".jpg", ".jpeg", ".png", ".bmp", ".webp", ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"]
LABELS = list(range(8))  # 0..7
BG = "bg"                # 라벨 없는 이미지용 가상 클래스

# ------------------------------------------------------------
# 유틸
# ------------------------------------------------------------
def ensure_dirs(root: Path):
    (root / "images").mkdir(exist_ok=True)
    (root / "labels").mkdir(exist_ok=True)
    for s, r in RATIOS.items():
        if r <= 0: 
            continue
        (root / "images" / s).mkdir(parents=True, exist_ok=True)
        (root / "labels" / s).mkdir(parents=True, exist_ok=True)

def parse_label_file(p: Path):
    """라벨 파일 안 등장한 클래스 집합 반환. 잘못된/빈 라인은 무시."""
    labs = set()
    try:
        with open(p, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    try:
                        cid = int(float(parts[0]))
                    except:
                        continue
                    if cid in LABELS:
                        labs.add(cid)
    except:
        pass
    return labs

def collect_items(ROOT: Path):
    """
    ROOT에서 목적지(images/labels 하위)를 제외하고 모든 이미지를 찾아
    (stem, img_path, lbl_path or None, labs_set) 리스트로 반환.
    라벨이 없거나 빈 라벨이면 labs_set={BG}
    """
    # 후보 이미지 수집
    candidates = []
    for ext in IMG_EXTS:
        for img in ROOT.rglob(f"*{ext}"):
            rel = img.relative_to(ROOT)
            if rel.parts and rel.parts[0] in ("images", "labels"):
                continue
            candidates.append(img)

    # 강한 섞기: 전역 셔플 + 블록 셔플
    stems = list({p.stem for p in candidates})
    random.shuffle(stems)
    if BLOCK_SIZE > 1:
        blocks = [stems[i:i+BLOCK_SIZE] for i in range(0, len(stems), BLOCK_SIZE)]
        for b in blocks:
            random.shuffle(b)
        random.shuffle(blocks)
        stems = [s for b in blocks for s in b]

    items = []
    for stem in stems:
        # ROOT 직하 우선, 없으면 재귀
        img = None
        for ext in IMG_EXTS:
            p = ROOT / f"{stem}{ext}"
            if p.exists():
                img = p; break
        if img is None:
            for ext in IMG_EXTS:
                hits = [h for h in ROOT.rglob(f"{stem}{ext}")
                        if (h.relative_to(ROOT).parts and h.relative_to(ROOT).parts[0] not in ("images","labels"))]
                if hits:
                    img = hits[0]; break
        if img is None:
            continue

        lbl = img.with_suffix(".txt")
        if not lbl.exists() or lbl.stat().st_size == 0:
            items.append((stem, img, None, {BG}))
        else:
            labs = parse_label_file(lbl)
            if labs:
                items.append((stem, img, lbl, labs))
            else:
                items.append((stem, img, None, {BG}))
    return items

# ------------------------------------------------------------
# 클래스 비율 유지 + 랜덤 노이즈 기반 층화 배치
# ------------------------------------------------------------
def balanced_interleave_stratify(items, labels=list(range(8)), bg_token="bg", seed=42):
    """
    items: [(stem, img_path, lbl_path or None, labs_set), ...]
    반환: {split: [item_index, ...]}
    """
    random.seed(seed)
    splits = list(RATIOS.keys())
    all_labels = labels + [bg_token]
    N = len(items)

    # 1) 사이즈 타깃(샘플 수)
    target_sizes = {s: int(round(N * RATIOS[s])) for s in splits}
    while sum(target_sizes.values()) < N:
        k = min(target_sizes, key=lambda x: target_sizes[x]); target_sizes[k] += 1
    while sum(target_sizes.values()) > N:
        k = max(target_sizes, key=lambda x: target_sizes[x]); target_sizes[k] -= 1

    # 2) 전역 라벨 분포 & 라벨 타깃
    global_counts = Counter()
    for _, _, _, labs in items:
        for l in labs:
            global_counts[l] += 1
    label_targets = {s: {l: global_counts[l] * RATIOS[s] for l in all_labels} for s in splits}

    # 상태
    split_sizes = {s: 0 for s in splits}
    split_label_counts = {s: Counter() for s in splits}
    assigned = {s: [] for s in splits}

    # split별 과충족 상한(라벨): 목표 * (1 + LABEL_TOL)
    label_caps = {s: {l: label_targets[s][l] * (1.0 + LABEL_TOL) for l in all_labels} for s in splits}

    # 아이템 순회 순서(강한 섞기)
    order = list(range(N))
    random.shuffle(order)
    if BLOCK_SIZE > 1:
        blocks = [order[i:i+BLOCK_SIZE] for i in range(0, N, BLOCK_SIZE)]
        for b in blocks:
            random.shuffle(b)
        random.shuffle(blocks)
        order = [i for b in blocks for i in b]

    def split_cost(s, labs):
        # 라벨 오차 변화
        label_term = 0.0
        for l in labs:
            cur = split_label_counts[s][l]
            tgt = label_targets[s][l]
            label_term += abs((cur + 1) - tgt) - abs(cur - tgt)
        # 사이즈 오차 변화
        size_term = abs((split_sizes[s] + 1) - target_sizes[s]) - abs(split_sizes[s] - target_sizes[s])
        # 무작위 노이즈(근접일 때 무작위로 갈라지게)
        noise = random.uniform(0.0, JITTER)
        return label_term + SIZE_WEIGHT * size_term + noise

    for idx in order:
        stem, img, lbl, labs = items[idx]

        # 1) 아직 사이즈 여유 있는 split만 1차 후보
        cand = [s for s in splits if split_sizes[s] < target_sizes[s]]
        if not cand:
            cand = splits[:]  # 전부 찼으면 전부 허용

        # 2) 라벨 과충족 제약(해당 아이템의 라벨 중 하나라도 cap을 넘기면 제외)
        feasible = []
        for s in cand:
            ok = True
            for l in labs:
                cap = label_caps[s].get(l, float("inf"))
                if split_label_counts[s][l] + 1 > cap and cap > 0:
                    ok = False
                    break
            if ok:
                feasible.append(s)
        if not feasible:
            feasible = cand  # 모두 막히면 완화

        # 3) 비용 최소 split 선택
        best_s, best_c = None, 1e18
        for s in feasible:
            c = split_cost(s, labs)
            if c < best_c:
                best_c = c
                best_s = s

        assigned[best_s].append(idx)
        split_sizes[best_s] += 1
        for l in labs:
            split_label_counts[best_s][l] += 1

    return assigned

# ------------------------------------------------------------
# 이동 & 리포트
# ------------------------------------------------------------
def move_into_structure(ROOT: Path, assigned, items):
    ensure_dirs(ROOT)

    def mv(src: Path, dst: Path):
        dst.parent.mkdir(parents=True, exist_ok=True)
        if src.resolve() == dst.resolve():
            return
        shutil.move(str(src), str(dst))

    for split, idxs in assigned.items():
        for i in idxs:
            stem, img, lbl, _ = items[i]
            dst_img = ROOT / "images" / split / img.name
            dst_lbl = ROOT / "labels" / split / f"{stem}.txt"
            mv(img, dst_img)
            if lbl and lbl.exists():
                mv(lbl, dst_lbl)

    # 간단 리포트
    counts = {s: len(idxs) for s, idxs in assigned.items()}
    print(f"  -> moved: train {counts.get('train',0)}, val {counts.get('val',0)}, test {counts.get('test',0)}")

def summarize_split(ROOT: Path):
    """각 split별 이미지 수·라벨 분포 요약 출력"""
    from collections import defaultdict
    def list_images(p: Path):
        return [x for x in p.rglob("*") if x.suffix in IMG_EXTS]

    for s in RATIOS.keys():
        img_dir = ROOT / "images" / s
        lbl_dir = ROOT / "labels" / s
        if not img_dir.exists():
            continue
        imgs = list_images(img_dir)
        stems_img = {p.stem for p in imgs}
        stems_lbl = {p.stem for p in lbl_dir.rglob("*.txt")} if lbl_dir.exists() else set()
        bg = len(stems_img - stems_lbl)

        cls_cnt = Counter()
        if lbl_dir.exists():
            for txt in lbl_dir.rglob("*.txt"):
                for line in txt.read_text(encoding="utf-8", errors="ignore").splitlines():
                    if not line.strip(): continue
                    parts = line.split()
                    try: cid = int(float(parts[0]))
                    except: continue
                    cls_cnt[cid] += 1

        print(f"  [{s}] images={len(imgs)}  BG(no-label)={bg}  objs={sum(cls_cnt.values())}  per-class={dict(sorted(cls_cnt.items()))}")

# ------------------------------------------------------------
# 실행
# ------------------------------------------------------------
if __name__ == "__main__":
    for root in TARGETS:
        print(f"\n=== Re-structuring in place: {root} ===")
        items = collect_items(root)
        if not items:
            print("  -> No images found. Skip.")
            continue
        assigned = balanced_interleave_stratify(items, labels=LABELS, bg_token=BG, seed=SEED)
        move_into_structure(root, assigned, items)
        summarize_split(root)
        print("✅ Done.")


In [ ]:
# segment_by_sequence_change.py
# - 순차 스캔 기반 컷 전환(클러스터 분할): 연속 프레임 간 변화가 커질 때 새 클러스터 시작
# - 변화량 = w1*(1-SSIM) + w2*HSV 히스토그램 거리 + w3*(1-ORB 매칭비)
# - 히스테리시스(THR_START > THR_CONT) + 최소 클러스터 길이로 안정화
# - ★ 최대 클러스터 길이 캡: MAX_CLUSTER_LEN 초과 시 강제 분할
# - 이동/분할 없음: 각 폴더별 cluster_map.tsv 저장 + 통계 출력

from pathlib import Path
from collections import Counter
import re
import cv2
import numpy as np

# ===== 대상 폴더 =====
TARGETS = [
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250925_merge_data/20250721_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250925_merge_data/20250725_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250925_merge_data/20250904_good_data"),
]

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp", ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}
SAVE_MAP = True

# ===== 데이터셋별 최대 클러스터 길이(선택) & 기본값 =====
# 키는 폴더명(root.name). 지정이 없으면 DEFAULT_MAX_LEN 사용.
PER_DATASET_MAX_LEN = {
    "20250721_good_data": 5,
    "20250725_good_data": 5,
    "20250904_good_data": 12,
}
DEFAULT_MAX_LEN = 10

# ===== 특징/거리 계산 파라미터 =====
RESIZE_WH = (512, 288)   # 비교용 리사이즈(가속 + 노이즈완화)
HIST_BINS = (32, 32, 32) # HSV 히스토그램 bin
HIST_RANGE = [0,180, 0,256, 0,256]
ORB_N = 800              # ORB keypoints 상한

# ===== 컷 전환(클러스터 분할) 파라미터 =====
W_SSIM = 0.55
W_HIST = 0.35
W_ORB  = 0.10            # 느리면 0.0으로 꺼도 됨

THR_START = 0.48         # 변화량이 이 값 이상이면 새 클러스터 시작
THR_CONT  = 0.38         # 클러스터 유지(전환 취소) 하한(히스테리시스)
MIN_CLUSTER_LEN = 1      # 너무 자주 끊기 방지

# 파일명 숫자 점프에 따른 보정(선택)
FNAME_NUM_PATTERN = re.compile(r"(\d+)")
NUM_JUMP_BOOST = 0.10
NUM_JUMP_K = 100

# -------------------------------
# 유틸
# -------------------------------
def natural_key(p: Path):
    s = str(p).lower()
    return [int(t) if t.isdigit() else t for t in re.findall(r'\d+|\D+', s)]

def list_images_sorted(ROOT: Path):
    cands = []
    for ext in IMG_EXTS:
        for img in ROOT.rglob(f"*{ext}"):
            rel = img.relative_to(ROOT)
            if rel.parts and rel.parts[0] in ("images", "labels"):
                continue
            cands.append(img)
    cands.sort(key=natural_key)
    return cands

def load_and_preprocess(p: Path):
    arr = cv2.imdecode(np.fromfile(str(p), dtype=np.uint8), cv2.IMREAD_COLOR)
    if arr is None:
        return None, None, None
    arr = cv2.resize(arr, RESIZE_WH, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(arr, cv2.COLOR_BGR2GRAY)
    hsv  = cv2.cvtColor(arr, cv2.COLOR_BGR2HSV)
    return arr, gray, hsv

def ssim_gray(a, b):
    C1 = (0.01*255)**2
    C2 = (0.03*255)**2
    a = a.astype(np.float32); b = b.astype(np.float32)
    mu1 = cv2.GaussianBlur(a, (11,11), 1.5)
    mu2 = cv2.GaussianBlur(b, (11,11), 1.5)
    mu1_sq = mu1*mu1; mu2_sq = mu2*mu2; mu12 = mu1*mu2
    sigma1_sq = cv2.GaussianBlur(a*a, (11,11), 1.5) - mu1_sq
    sigma2_sq = cv2.GaussianBlur(b*b, (11,11), 1.5) - mu2_sq
    sigma12 = cv2.GaussianBlur(a*b, (11,11), 1.5) - mu12
    ssim_map = ((2*mu12 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2) + 1e-8)
    return float(np.clip(ssim_map.mean(), -1.0, 1.0))

def hist_dist_hsv(hsv1, hsv2):
    h1 = cv2.calcHist([hsv1],[0,1,2],None,HIST_BINS,HIST_RANGE)
    h2 = cv2.calcHist([hsv2],[0,1,2],None,HIST_BINS,HIST_RANGE)
    cv2.normalize(h1, h1, 1, 0, cv2.NORM_L1)
    cv2.normalize(h2, h2, 1, 0, cv2.NORM_L1)
    sim = cv2.compareHist(h1, h2, cv2.HISTCMP_BHATTACHARYYA)
    return float(np.clip(sim, 0.0, 1.0))

def orb_mismatch_ratio(img1, img2):
    orb = cv2.ORB_create(nfeatures=ORB_N)
    k1, d1 = orb.detectAndCompute(img1, None)
    k2, d2 = orb.detectAndCompute(img2, None)
    if d1 is None or d2 is None or len(k1) < 10 or len(k2) < 10:
        return 1.0
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(d1, d2)
    if not matches:
        return 1.0
    matches = sorted(matches, key=lambda m: m.distance)
    keep = matches[:max(20, len(matches)//2)]
    good = sum(1 for m in keep if m.distance <= 70)
    ratio = good / max(1, len(keep))
    return float(1.0 - ratio)

def fname_num(p: Path):
    m = FNAME_NUM_PATTERN.findall(p.stem)
    if not m: return None
    try: return int(m[-1])
    except: return None

def change_score(prev, curr):
    b1, g1, h1 = prev
    b2, g2, h2 = curr
    ssim = ssim_gray(g1, g2)
    d_ssim = 1.0 - ((ssim + 1.0) / 2.0)
    d_hist = hist_dist_hsv(h1, h2)
    d_orb = orb_mismatch_ratio(b1, b2) if W_ORB > 0.0 else 0.0
    return float(W_SSIM*d_ssim + W_HIST*d_hist + W_ORB*d_orb)

# -------------------------------
# 순차 컷 전환 분할 (+ 최대 길이 캡)
# -------------------------------
def segment_sequence(images, max_cluster_len):
    preps = []
    nums  = []
    for p in images:
        bgr, gray, hsv = load_and_preprocess(p)
        preps.append((bgr, gray, hsv))
        nums.append(fname_num(p))

    clusters = []
    cur = []
    state_high = False

    def flush():
        if cur:
            clusters.append(cur.copy())
            cur.clear()

    for i in range(len(images)):
        if preps[i][0] is None:
            flush()
            clusters.append([i])
            state_high = False
            continue

        if not cur:
            cur.append(i)
            continue

        # 최대 길이 캡 우선 적용: 초과 직전에 컷
        if len(cur) >= max_cluster_len:
            flush()
            cur.append(i)
            state_high = False
            continue

        # 변화량 계산(i-1 -> i)
        score = change_score(preps[i-1], preps[i])

        # 파일명 숫자 점프 보정
        if nums[i-1] is not None and nums[i] is not None:
            if nums[i] - nums[i-1] > NUM_JUMP_K:
                score += NUM_JUMP_BOOST

        if not state_high:
            if score >= THR_START and len(cur) >= MIN_CLUSTER_LEN:
                flush()
                cur.append(i)
                state_high = True
            else:
                cur.append(i)
        else:
            # 전환 상태 해제
            if score <= THR_CONT:
                state_high = False
            # 새 클러스터 시작 상태이므로 현재 i는 이미 cur에 포함됨

    flush()
    return clusters

def print_stats(ROOT: Path, images, clusters, max_len_used):
    n_items = len(images)
    n_cl = len(clusters)
    avg_sz = n_items / max(1, n_cl)
    size_counts = Counter(len(c) for c in clusters)
    buckets = [(i, size_counts.get(i, 0)) for i in range(1, 11)]
    ge11 = sum(v for k, v in size_counts.items() if k >= 11)

    print(f"\n=== Sequential change-based clusters: {ROOT} ===")
    print(f"  -> MAX_CLUSTER_LEN = {max_len_used}")
    print(f"  -> items: {n_items}, clusters: {n_cl} (avg size ~ {avg_sz:.2f})")
    line = "  -> size histogram: " + "  ".join([f"{k}:{v}" for k, v in buckets])
    if ge11: line += f"  11+:{ge11}"
    print(line)

    top = sorted(range(n_cl), key=lambda c: -len(clusters[c]))[:5]
    for rank, cid in enumerate(top, 1):
        sz = len(clusters[cid])
        samples = [str(images[i].relative_to(ROOT)) for i in clusters[cid][:3]]
        print(f"  -> top#{rank}: cid={cid+1:03d}, size={sz}, samples={samples}")

    if SAVE_MAP:
        out = ROOT / "cluster_map.tsv"
        with out.open("w", encoding="utf-8") as f:
            f.write("cluster_id\timage_relpath\n")
            for cid, cl in enumerate(clusters, start=1):
                for idx in cl:
                    rel = images[idx].relative_to(ROOT)
                    f.write(f"{cid}\t{rel}\n")
        print(f"  -> saved: {out}")

# -------------------------------
# 실행
# -------------------------------
if __name__ == "__main__":
    for root in TARGETS:
        imgs = list_images_sorted(root)
        if not imgs:
            print(f"\n=== Sequential change-based clusters: {root} ===")
            print("  -> No images found. Skip.")
            continue

        # 데이터셋별 최대 길이 선택
        max_len = PER_DATASET_MAX_LEN.get(root.name, DEFAULT_MAX_LEN)
        clusters = segment_sequence(imgs, max_len)
        print_stats(root, imgs, clusters, max_len)

    print("\n✅ Analysis only (max cluster length cap enabled)")



### 클러스터 csv 만들기 _ by LBP + ZNCC

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
순차 스캔: 조명 영향 제거(밝기/대비 불변) 기준 + 드리프트 억제(앵커 비교) + 폴더별 최대 길이 제한
- 파일명 자연정렬
- LBP 히스토그램 + ZNCC 사용 (밝기/대비 변화에 둔감)
- change = GAMMA * d_lbp + (1-GAMMA) * (1-zncc)
- (prev 또는 anchor 기준 change) >= CHANGE_THRESH 이면 새 그룹
- 폴더별 파라미터 오버라이드(FOLDER_CFG) + 폴더별 MAX_CLUSTER_LEN
- MAX_CLUSTER_LEN 도달 시 유사도와 무관하게 새 클러스터 시작(앵커 갱신)

CSV: clusters_invariant_<basename>.csv
  filename, cluster_id, index_in_folder,
  d_lbp_prev, zncc_prev, change_prev,
  d_lbp_anchor, zncc_anchor, change_anchor,
  is_cluster_start, cluster_len_so_far, reason
"""

from pathlib import Path
import cv2
import numpy as np
import csv, re
from typing import Optional, List, Dict, Tuple

# ====== 대상 폴더 ======
FOLDERS = [
    #Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250721_good_data"),
    #Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250725_good_data"),
    #Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250904_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250930_good_data"),
]

# ====== 폴더별 파라미터(없으면 DEFAULT_CFG 사용) ======
# - GAMMA: LBP 비중(0~1). 낮출수록 ZNCC 비중↑ → 구조 차이에 더 민감
# - CHANGE_THRESH: 컷 전환 임계 (낮을수록 더 잘 끊김)
# - GAUSS_BLUR: LBP/ZNCC 전처리 블러 커널 (작을수록 민감)
# - ZNCC_SIZE: ZNCC 계산 리사이즈 한 변
# - MAX_CLUSTER_LEN: 현재 클러스터 최대 길이(초과 시 강제 분리). 예: 3이면 3장까지 같은 그룹.
FOLDER_CFG: Dict[str, Dict] = {
    "20250721_good_data": {
        "GAMMA": 0.60,
        "CHANGE_THRESH": 0.32,
        "GAUSS_BLUR": (3,3),
        "ZNCC_SIZE": 256,
        "MAX_CLUSTER_LEN": 5,
    },
    "20250725_good_data": {
        "GAMMA": 0.60,
        "CHANGE_THRESH": 0.35,
        "GAUSS_BLUR": (3,3),
        "ZNCC_SIZE": 256,
        "MAX_CLUSTER_LEN": 5,
    },
    "20250904_good_data": {
        "GAMMA": 0.55,
        "CHANGE_THRESH": 0.25,
        "GAUSS_BLUR": (3,3),
        "ZNCC_SIZE": 256,
        "MAX_CLUSTER_LEN": 20,  # 예시: 12장까지만 같은 그룹
    },
    "20250930_good_data": {
        "GAMMA": 0.60,
        "CHANGE_THRESH": 0.28,
        "GAUSS_BLUR": (3,3),
        "ZNCC_SIZE": 256,
        "MAX_CLUSTER_LEN": 15,  # 예시: 12장까지만 같은 그룹
    },
}

# 기본값 (위에 없으면 이 값을 사용)
DEFAULT_CFG = {
    "GAMMA": 0.60,
    "CHANGE_THRESH": 0.20,
    "GAUSS_BLUR": (3,3),
    "ZNCC_SIZE": 256,
    "MAX_CLUSTER_LEN": 5,
}

# ====== 이미지 확장자 ======
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp",
            ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}

# ====== LBP 설정 ======
LBP_RADIUS = 1
LBP_POINTS = 8
LBP_HIST_BINS = 256  # 8-bit LBP

# ====== 기타 ======
NATURAL_SORT = True
MAX_IMAGES = None
DOWNSCALE_IF_NEEDED = False
PREPROC_MAX_SIDE = 0   # 0=축소 안함(정확도 우선)


# ---------------- 유틸 ----------------
def get_cfg(folder: Path) -> Dict:
    return {**DEFAULT_CFG, **FOLDER_CFG.get(folder.name, {})}

def natural_key(s: str):
    if not NATURAL_SORT: return s
    return [int(t) if t.isdigit() else t.lower() for t in re.split(r"(\d+)", s)]

def list_images(folder: Path) -> List[Path]:
    files = [p for p in folder.iterdir() if p.is_file() and p.suffix in IMG_EXTS]
    files.sort(key=lambda p: natural_key(p.name))
    return files[:MAX_IMAGES] if MAX_IMAGES else files

def resize_longest(img: np.ndarray, max_side: int) -> np.ndarray:
    if max_side is None or max_side <= 0: return img
    h, w = img.shape[:2]
    longest = max(h, w)
    if longest <= max_side: return img
    scale = max_side / float(longest)
    return cv2.resize(img, (int(round(w*scale)), int(round(h*scale))), interpolation=cv2.INTER_AREA)

def load_bgr(path: Path) -> Optional[np.ndarray]:
    img = cv2.imread(str(path), cv2.IMREAD_COLOR)
    if img is None: return None
    if DOWNSCALE_IF_NEEDED and PREPROC_MAX_SIDE > 0:
        img = resize_longest(img, PREPROC_MAX_SIDE)
    return img


# ---------------- LBP & ZNCC ----------------
def lbp_image(gray: np.ndarray) -> np.ndarray:
    """간단 3x3 LBP (8-bit)."""
    g = gray
    c = g[1:-1, 1:-1]
    code = np.zeros_like(c, dtype=np.uint8)
    nbrs = [
        (0, 0),  (0, 1),  (0, 2),
        (1, 2),  (2, 2),  (2, 1),
        (2, 0),  (1, 0),
    ]
    for bit, (dy, dx) in enumerate(nbrs):
        n = g[dy:dy + c.shape[0], dx:dx + c.shape[1]]
        code |= ((n >= c) << (7 - bit)).astype(np.uint8)
    return code

def lbp_hist(gray: np.ndarray, blur_kernel: Tuple[int,int]) -> np.ndarray:
    """정규화된 LBP 히스토그램(밝기/대비 변화에 둔감)."""
    if blur_kernel and blur_kernel[0] > 0 and blur_kernel[1] > 0:
        gray = cv2.GaussianBlur(gray, blur_kernel, 0)
    lbp = lbp_image(gray)
    hist = cv2.calcHist([lbp], [0], None, [LBP_HIST_BINS], [0, 256])
    cv2.normalize(hist, hist)
    return hist

def d_bhat(h1: np.ndarray, h2: np.ndarray) -> float:
    return float(cv2.compareHist(h1, h2, cv2.HISTCMP_BHATTACHARYYA))

def zncc(gray1: np.ndarray, gray2: np.ndarray, size: int, blur_kernel: Tuple[int,int]) -> float:
    """Zero-mean NCC (밝기/대비 선형 변화 불변). -1~1, 1=완전 동일"""
    g1 = cv2.resize(gray1, (size, size), interpolation=cv2.INTER_AREA)
    g2 = cv2.resize(gray2, (size, size), interpolation=cv2.INTER_AREA)
    if blur_kernel and blur_kernel[0] > 0 and blur_kernel[1] > 0:
        g1 = cv2.GaussianBlur(g1, blur_kernel, 0)
        g2 = cv2.GaussianBlur(g2, blur_kernel, 0)
    g1 = g1.astype(np.float32)
    g2 = g2.astype(np.float32)
    g1 -= g1.mean(); g2 -= g2.mean()
    std1 = g1.std() + 1e-6
    std2 = g2.std() + 1e-6
    g1 /= std1; g2 /= std2
    num = float(np.sum(g1 * g2))
    den = float(np.sqrt(np.sum(g1*g1)) * np.sqrt(np.sum(g2*g2)) + 1e-6)
    return float(np.clip(num / den, -1.0, 1.0))

def gray_of(img_bgr: np.ndarray) -> np.ndarray:
    return cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)


# ---------------- 메인 ----------------
def cluster_folder(folder: Path):
    cfg = get_cfg(folder)
    GAMMA = float(cfg["GAMMA"])
    CHANGE_THRESH = float(cfg["CHANGE_THRESH"])
    GAUSS_BLUR = tuple(cfg["GAUSS_BLUR"]) if isinstance(cfg["GAUSS_BLUR"], (list, tuple)) else (3,3)
    ZNCC_SIZE = int(cfg["ZNCC_SIZE"])
    MAX_CLUSTER_LEN = int(cfg.get("MAX_CLUSTER_LEN", 0))  # 0 또는 음수면 제한 없음

    imgs = list_images(folder)
    if not imgs:
        print(f"[WARN] No images in: {folder}")
        return

    img_prev = load_bgr(imgs[0])
    if img_prev is None:
        print(f"[WARN] Can't read: {imgs[0]}")
        return

    g_prev = gray_of(img_prev)
    lbp_prev = lbp_hist(g_prev, GAUSS_BLUR)

    # 앵커(그룹 첫 프레임)
    anchor_img = img_prev
    g_anchor = g_prev
    lbp_anchor = lbp_prev

    cluster_id = 0
    curr_len = 1  # 현재 클러스터 길이(첫 프레임 포함)

    rows = [{
        "filename": imgs[0].name,
        "cluster_id": cluster_id,
        "index_in_folder": 0,
        "d_lbp_prev": 0.0,
        "zncc_prev": 1.0,
        "change_prev": 0.0,
        "d_lbp_anchor": 0.0,
        "zncc_anchor": 1.0,
        "change_anchor": 0.0,
        "is_cluster_start": 1,
        "cluster_len_so_far": curr_len,
        "reason": "start",
    }]

    for idx in range(1, len(imgs)):
        path = imgs[idx]
        img = load_bgr(path)
        if img is None:
            print(f"[WARN] Can't read: {path}")
            continue

        g_cur = gray_of(img)
        lbp_cur = lbp_hist(g_cur, GAUSS_BLUR)

        # prev 기준
        d_prev = d_bhat(lbp_prev, lbp_cur)
        z_prev = zncc(g_prev, g_cur, ZNCC_SIZE, GAUSS_BLUR)
        change_prev = GAMMA * d_prev + (1.0 - GAMMA) * (1.0 - z_prev)

        # anchor 기준
        d_anchor = d_bhat(lbp_anchor, lbp_cur)
        z_anchor = zncc(g_anchor, g_cur, ZNCC_SIZE, GAUSS_BLUR)
        change_anchor = GAMMA * d_anchor + (1.0 - GAMMA) * (1.0 - z_anchor)

        # 길이 제한 체크 (현재 클러스터가 이미 MAX에 도달했는지)
        limit_hit = (MAX_CLUSTER_LEN > 0) and (curr_len >= MAX_CLUSTER_LEN)

        # 컷 판정: 길이 제한 우선, 아니면 변화 기준
        if limit_hit or (change_prev >= CHANGE_THRESH) or (change_anchor >= CHANGE_THRESH):
            cluster_id += 1
            is_start = 1
            reason = "limit" if limit_hit else "new"
            curr_len = 1  # 새 클러스터 길이 초기화
            # 앵커 갱신
            anchor_img = img
            g_anchor = g_cur
            lbp_anchor = lbp_cur
        else:
            is_start = 0
            reason = "similar"
            curr_len += 1

        rows.append({
            "filename": path.name,
            "cluster_id": cluster_id,
            "index_in_folder": idx,
            "d_lbp_prev": round(d_prev, 6),
            "zncc_prev": round(z_prev, 6),
            "change_prev": round(change_prev, 6),
            "d_lbp_anchor": round(d_anchor, 6),
            "zncc_anchor": round(z_anchor, 6),
            "change_anchor": round(change_anchor, 6),
            "is_cluster_start": is_start,
            "cluster_len_so_far": curr_len,
            "reason": reason,
        })

        # 다음 비교 기준 업데이트
        img_prev = img
        g_prev = g_cur
        lbp_prev = lbp_cur

    out_csv = folder.parent / f"clusters_invariant_{folder.name}.csv"
    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=[
            "filename","cluster_id","index_in_folder",
            "d_lbp_prev","zncc_prev","change_prev",
            "d_lbp_anchor","zncc_anchor","change_anchor",
            "is_cluster_start","cluster_len_so_far","reason"
        ])
        w.writeheader(); w.writerows(rows)

    n_clusters = (rows[-1]["cluster_id"] if rows else -1) + 1
    print(f"[OK] {folder.name}: {len(rows)} images → {n_clusters} clusters "
          f"(GAMMA={GAMMA}, TH={CHANGE_THRESH}, BLUR={GAUSS_BLUR}, ZNCC={ZNCC_SIZE}, MAXLEN={MAX_CLUSTER_LEN}) "
          f"→ {out_csv}")

def main():
    for folder in FOLDERS:
        if not folder.exists():
            print(f"[WARN] Not found: {folder}")
            continue
        cluster_folder(folder)

if __name__ == "__main__":
    main()


### csv로 해당폴더안에 있는 파일들 이름 바꾸기

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
import csv
import re

# ===== 설정 =====
CSV_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data")
CSV_GLOB = "clusters_invariant_*.csv"
IMG_ROOT = CSV_DIR

APPLY_CHANGES = True # 실제 적용하려면 True

# suffix: "_cluster_<id>"
def make_suffix(cluster_id: int) -> str:
    return f"_cluster_{cluster_id}"

# 이미 붙어있는지 체크 (basename 끝이 _cluster_숫자)
SUFFIX_RE = re.compile(r"_cluster_(\d+)$", re.IGNORECASE)

def with_seq(p: Path, k: int) -> Path:
    return p.with_name(f"{p.stem}__{k}{p.suffix}")

def parse_folder_from_csv(csv_path: Path) -> Path:
    m = re.match(r"clusters_invariant_(.+)\.csv$", csv_path.name)
    if not m:
        raise ValueError(f"CSV 이름 형식이 아님: {csv_path.name}")
    folder_name = m.group(1)
    return IMG_ROOT / folder_name

def ensure_nonconflicting(target: Path) -> Path:
    if not target.exists():
        return target
    k = 2
    cand = with_seq(target, k)
    while cand.exists():
        k += 1
        cand = with_seq(target, k)
    return cand

def plan_for_csv(csv_path: Path):
    folder = parse_folder_from_csv(csv_path)
    if not folder.exists():
        print(f"[WARN] 폴더 없음: {folder}  (CSV={csv_path.name})")
        return []

    plans = []  # list of dicts: {old_jpg, new_jpg, old_txt, new_txt}
    with csv_path.open("r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        if "filename" not in reader.fieldnames or "cluster_id" not in reader.fieldnames:
            raise ValueError(f"[ERR] CSV에 filename/cluster_id 없음: {csv_path}")

        for row in reader:
            fn = row["filename"]
            cid = int(row["cluster_id"])

            # jpg만 처리
            if not fn.lower().endswith(".jpg"):
                continue

            jpg_path = folder / fn
            if not jpg_path.exists():
                continue

            stem = jpg_path.stem
            # 이미 _cluster_xxx 붙어있으면 스킵
            if SUFFIX_RE.search(stem):
                continue

            txt_path = folder / f"{stem}.txt"
            suffix = make_suffix(cid)
            new_stem = f"{stem}{suffix}"

            new_jpg = jpg_path.with_name(f"{new_stem}{jpg_path.suffix}")
            new_txt = (txt_path.with_name(f"{new_stem}{txt_path.suffix}")
                       if txt_path.exists() else None)

            plans.append({
                "old_jpg": jpg_path,
                "new_jpg": new_jpg,
                "old_txt": txt_path if txt_path.exists() else None,
                "new_txt": new_txt
            })
    return plans

def run():
    csvs = sorted(CSV_DIR.glob(CSV_GLOB))
    if not csvs:
        print(f"[WARN] CSV 없음: {CSV_DIR}/{CSV_GLOB}")
        return

    total_sets = 0
    for csv_path in csvs:
        plans = plan_for_csv(csv_path)
        if not plans:
            print(f"[INFO] 변경 계획 없음: {csv_path.name}")
            continue

        folder = parse_folder_from_csv(csv_path)
        print(f"\n[PLAN] {csv_path.name} → 폴더: {folder}")
        for p in plans:
            old_jpg, new_jpg = p["old_jpg"], p["new_jpg"]
            old_txt, new_txt = p["old_txt"], p["new_txt"]

            safe_new_jpg = ensure_nonconflicting(new_jpg)
            safe_new_txt = ensure_nonconflicting(new_txt) if new_txt else None

            # 출력(계획)
            if old_txt and safe_new_txt:
                print(f"  - JPG: {old_jpg.name} → {safe_new_jpg.name}")
                print(f"    TXT: {old_txt.name} → {safe_new_txt.name}")
            else:
                print(f"  - JPG: {old_jpg.name} → {safe_new_jpg.name} (txt 없음, jpg만 변경)")

            if APPLY_CHANGES:
                # txt가 있으면 반드시 같이 변경
                if old_txt and safe_new_txt:
                    old_txt.rename(safe_new_txt)
                    # txt 변경 후 jpg 타겟 충돌 재확인
                    safe_new_jpg = ensure_nonconflicting(safe_new_jpg)
                    old_jpg.rename(safe_new_jpg)
                else:
                    # txt가 없으면 jpg만 변경
                    old_jpg.rename(safe_new_jpg)
                total_sets += 1

    if APPLY_CHANGES:
        print(f"\n[OK] 실제 변경 완료. 총 {total_sets}건 처리.")
    else:
        print("\n[DRY-RUN] 실제 변경 없음. APPLY_CHANGES=True 로 적용하세요.")

if __name__ == "__main__":
    run()

### 데이터 분할하기_1 일단 0 1 2 위주로 cluster별로 분할

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
import re
import random
import shutil

# ===== 설정 =====
FOLDERS = [
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250721_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250725_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250904_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250930_good_data"),
]

# 리포트 저장 디렉터리(각 폴더의 상위 경로)
REPORT_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data")

# split 비율 (0=train, 1=val, 2=test)
RATIOS = {0: 0.70, 1: 0.18, 2: 0.12}
SPLIT_NAME = {0: "train", 1: "val", 2: "test"}
SEED = 13

# 분포 유지(균형)에 쓸 클래스 / 리포팅 클래스
BALANCE_CLASS_IDS = [0, 1, 2]         # 균형 유지용
ALL_CLASS_IDS = list(range(8))        # 리포트 출력용

# 입력 파일 패턴 (_cluster_<id> 전제)
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp",
            ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}
CLUSTER_RE = re.compile(r"^(?P<stem>.+)_cluster_(?P<cid>\d+)(?:__\d+)?$", re.IGNORECASE)

# 실제 분할 수행
DO_MATERIALIZE = True
# "move" | "copy" | "symlink"
MATERIALIZE_MODE = "move"
# 기존 images/labels 내 train/val/test 디렉터리 있으면 비우고 다시 만들기
CLEAN_PREVIOUS = True
# 라벨이 없을 때 빈 txt를 생성할지 여부 (False => 생성하지 않음)
CREATE_EMPTY_LABEL_IF_MISSING = False

# 스코어 가중치: 이미지/클래스/bg(무라벨 이미지 개수)
IMG_W = 1.0
CLS_W = 1.0
BG_W  = 1.0

# ---------- 유틸 ----------
def find_images(folder: Path):
    # 루트에 섞여있는 이미지만 스캔 (이미 분할된 하위 디렉토리는 무시)
    return [p for p in folder.iterdir() if p.is_file() and p.suffix in IMG_EXTS]

def parse_cluster_id(stem: str):
    m = CLUSTER_RE.match(stem)
    if not m:
        return None, None
    return m.group("stem"), int(m.group("cid"))

def read_label_lines(txt_path: Path):
    if not txt_path.exists():
        return []
    try:
        with txt_path.open("r", encoding="utf-8") as f:
            return [ln.strip() for ln in f if ln.strip()]
    except Exception:
        return []

def count_classes_from_txt_lines(lines, class_ids):
    counts = {k: 0 for k in class_ids}
    for s in lines:
        parts = s.split()
        try:
            cid = int(parts[0])
        except Exception:
            continue
        if cid in counts:
            counts[cid] += 1
    return counts

def build_cluster_stats(folder: Path):
    """
    clusters[cid] = {
        "images": [Path(img1), ...],
        "class_counts_bal": {0:x,1:y,2:z},  # 분할 균형용
        "class_counts_all": {0..7: n},      # 리포팅용
        "bg_count": int,                    # ★ 무라벨 이미지 수 (파일 없음 또는 라인 0)
    }
    """
    clusters = {}
    for img in find_images(folder):
        _, cid = parse_cluster_id(img.stem)
        if cid is None:
            # _cluster_<id> 없는 파일은 분할 대상에서 제외
            continue

        txt = img.with_suffix(".txt")
        lines = read_label_lines(txt)
        counts_all = count_classes_from_txt_lines(lines, ALL_CLASS_IDS)
        counts_bal = {k: counts_all.get(k, 0) for k in BALANCE_CLASS_IDS}
        is_bg = (len(lines) == 0)  # ★ 파일이 없거나 라인이 0이면 bg로 간주

        if cid not in clusters:
            clusters[cid] = {
                "images": [],
                "class_counts_bal": {k: 0 for k in BALANCE_CLASS_IDS},
                "class_counts_all": {k: 0 for k in ALL_CLASS_IDS},
                "bg_count": 0,
            }
        clusters[cid]["images"].append(img)
        for k in BALANCE_CLASS_IDS:
            clusters[cid]["class_counts_bal"][k] += counts_bal[k]
        for k in ALL_CLASS_IDS:
            clusters[cid]["class_counts_all"][k] += counts_all[k]
        clusters[cid]["bg_count"] += 1 if is_bg else 0

    return clusters

def stratified_split_for_folder(clusters: dict, ratios: dict, balance_ids: list, seed=42):
    """
    폴더 하나에 대해 클러스터 단위로 train/val/test 배정.
    목표: 전체 이미지 수 + class 0/1/2 분포 + bg(무라벨 이미지 수)가 ratios에 근접.
    방법: 큰 클러스터부터 하나씩 배치하며 가중 잔차^2 증가 최소 split 선택.
    """
    random.seed(seed)

    cluster_items = []
    total_imgs = 0
    total_cls = {k: 0 for k in balance_ids}
    total_bg = 0

    for cid, info in clusters.items():
        sz = len(info["images"])
        total_imgs += sz
        vec = {"imgs": sz, "bg": info.get("bg_count", 0)}
        total_bg += vec["bg"]
        for k in balance_ids:
            vec[k] = info["class_counts_bal"][k]
            total_cls[k] += vec[k]
        cluster_items.append((cid, vec))

    if total_imgs == 0:
        return {}

    targets_imgs = {s: ratios[s] * total_imgs for s in ratios}
    targets_bg   = {s: ratios[s] * total_bg   for s in ratios}
    targets_class = {k: {s: ratios[s] * total_cls[k] for s in ratios} for k in balance_ids}

    # 큰 클러스터부터
    cluster_items.sort(key=lambda x: (-x[1]["imgs"], x[0]))

    assigned_imgs = {s: 0.0 for s in ratios}
    assigned_bg   = {s: 0.0 for s in ratios}
    assigned_class = {k: {s: 0.0 for s in ratios} for k in balance_ids}
    mapping = {}

    def score_components(ai_imgs, ai_bg, ai_cls):
        img_resid = sum((ai_imgs[s] - targets_imgs[s])**2 for s in ratios)
        bg_resid  = sum((ai_bg[s]   - targets_bg[s])**2   for s in ratios)
        cls_resid = 0.0
        for k in balance_ids:
            cls_resid += sum((ai_cls[k][s] - targets_class[k][s])**2 for s in ratios)
        # 가중합
        return IMG_W * img_resid + BG_W * bg_resid + CLS_W * cls_resid

    def score_after_add(split: int, vec: dict):
        ai_imgs = {s: assigned_imgs[s] + (vec["imgs"] if s == split else 0.0) for s in ratios}
        ai_bg   = {s: assigned_bg[s]   + (vec["bg"]   if s == split else 0.0) for s in ratios}
        ai_cls  = {k: {s: assigned_class[k][s] + (vec[k] if s == split else 0.0) for s in ratios}
                   for k in balance_ids}
        return score_components(ai_imgs, ai_bg, ai_cls)

    for cid, vec in cluster_items:
        best_s, best_sc = None, None
        for s in ratios:
            sc = score_after_add(s, vec)
            if best_sc is None or sc < best_sc:
                best_sc, best_s = sc, s
        mapping[cid] = best_s
        assigned_imgs[best_s] += vec["imgs"]
        assigned_bg[best_s]   += vec["bg"]
        for k in balance_ids:
            assigned_class[k][best_s] += vec[k]

    return mapping

# --- 여기부터 출력 디렉토리 구조: images/labels 안에 train|val|test ---
def ensure_clean_split_dirs(base: Path):
    """
    <base>/images/train|val|test
    <base>/labels/train|val|test
    """
    images_root = base / "images"
    labels_root = base / "labels"
    if CLEAN_PREVIOUS:
        if images_root.exists(): shutil.rmtree(images_root, ignore_errors=True)
        if labels_root.exists(): shutil.rmtree(labels_root, ignore_errors=True)
    for split in ("train", "val", "test"):
        (images_root / split).mkdir(parents=True, exist_ok=True)
        (labels_root / split).mkdir(parents=True, exist_ok=True)

def place_file(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    if MATERIALIZE_MODE == "move":
        shutil.move(str(src), str(dst))
    elif MATERIALIZE_MODE == "copy":
        shutil.copy2(str(src), str(dst))
    elif MATERIALIZE_MODE == "symlink":
        try:
            dst.symlink_to(src.resolve())
        except FileExistsError:
            dst.unlink()
            dst.symlink_to(src.resolve())
    else:
        raise ValueError("MATERIALIZE_MODE must be 'move'|'copy'|'symlink'.")

def materialize_and_save_report(folder: Path, clusters: dict, mapping: dict, report_dir: Path):
    """
    YOLO 구조로 실제 배치 (images/labels 아래 train|val|test) +
    split별 클래스 분포(0~7) 콘솔 출력 & 텍스트 저장
    """
    ensure_clean_split_dirs(folder)

    split_img_counts = {0:0, 1:0, 2:0}
    split_bg_counts  = {0:0, 1:0, 2:0}  # ★ bg 이미지 수 리포팅
    split_class_counts = {s: {k:0 for k in ALL_CLASS_IDS} for s in (0,1,2)}

    for cid, info in clusters.items():
        sp = mapping[cid]
        img_dir = folder / "images" / SPLIT_NAME[sp]
        lbl_dir = folder / "labels" / SPLIT_NAME[sp]

        # 리포트 집계
        for k in ALL_CLASS_IDS:
            split_class_counts[sp][k] += info["class_counts_all"][k]
        split_bg_counts[sp] += info.get("bg_count", 0)

        for img in info["images"]:
            split_img_counts[sp] += 1

            # 이미지 배치
            img_dst = img_dir / img.name
            place_file(img, img_dst)

            # 라벨 배치 (없으면 정책에 따라 생성/미생성)
            txt_src = img.with_suffix(".txt")
            txt_dst = lbl_dir / txt_src.name
            if txt_src.exists():
                place_file(txt_src, txt_dst)
            elif CREATE_EMPTY_LABEL_IF_MISSING:
                txt_dst.parent.mkdir(parents=True, exist_ok=True)
                txt_dst.touch(exist_ok=True)
            # else: 생성하지 않음

    # ----- 리포트 구성 -----
    lines = []
    total = sum(split_img_counts.values())
    header = f"[OK] {folder.name} 분할 완료 (총 {total} images)  mode={MATERIALIZE_MODE}"
    lines.append(header)
    for s in (0,1,2):
        cnt = split_img_counts[s]
        pct = (cnt/total)*100 if total else 0
        lines.append(f"  {SPLIT_NAME[s]}: {cnt} imgs ({pct:.1f}%)")

    # bg 분포도 추가 출력
    total_bg = sum(split_bg_counts.values())
    lines.append("\n[BG (no-label) image distribution]")
    for s in (0,1,2):
        pct = (split_bg_counts[s]/total_bg*100) if total_bg else 0.0
        lines.append(f"  {SPLIT_NAME[s]}: {split_bg_counts[s]} bg ({pct:.1f}%)")

    lines.append("\n[Class distribution (instances per split)]")
    hdr = " ".join([f"{SPLIT_NAME[s]:>10}" for s in (0,1,2)])
    lines.append(f"{'class':>10} {hdr}")
    for k in ALL_CLASS_IDS:
        row = " ".join([f"{split_class_counts[s][k]:>10d}" for s in (0,1,2)])
        lines.append(f"{k:>10} {row}")

    report_dir.mkdir(parents=True, exist_ok=True)
    out_txt = report_dir / f"split_report_{folder.name}.txt"
    with out_txt.open("w", encoding="utf-8") as f:
        f.write("\n".join(lines) + "\n")

    print("\n" + "\n".join(lines))
    print(f"\n[Saved] {out_txt}")

def run():
    random.seed(SEED)
    REPORT_DIR.mkdir(parents=True, exist_ok=True)

    for folder in FOLDERS:
        if not folder.exists():
            print(f"[WARN] 폴더 없음: {folder}")
            continue

        # 1) 폴더 루트에서 _cluster_<id> 파일들만 모아 통계 (+ bg_count 포함)
        clusters = build_cluster_stats(folder)
        if not clusters:
            print(f"[INFO] 클러스터 없음(_cluster_<id> 파일 미존재): {folder.name}")
            continue

        # 2) 폴더 단위 분할 (클러스터 유지 + class 0/1/2 + bg 분포 유지)
        mapping = stratified_split_for_folder(clusters, RATIOS, BALANCE_CLASS_IDS, seed=SEED)

        # 3) 실제 YOLO 구조로 배치(images/labels 밑에 split) + 리포트 저장
        if DO_MATERIALIZE:
            materialize_and_save_report(folder, clusters, mapping, REPORT_DIR)
        else:
            print(f"[DRY] 분할 매핑만 완료(배치 미수행): {folder.name}")

if __name__ == "__main__":
    run()


### 섞였는지 점검

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from pathlib import Path
import re
from collections import defaultdict

# ===== 점검할 데이터 폴더들 =====
FOLDERS = [
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250721_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250725_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250904_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250930_good_data"),
]

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp",
            ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}
CLUSTER_RE = re.compile(r"_cluster_(\d+)", re.IGNORECASE)

def check_folder(folder: Path):
    print(f"\n=== Checking {folder.name} ===")
    # cluster_id -> set of splits where it appears
    cluster_splits = defaultdict(set)

    for split in ("train", "val", "test"):
        img_dir = folder / "images" / split
        if not img_dir.exists():
            continue
        for p in img_dir.rglob("*"):
            if p.is_file() and p.suffix in IMG_EXTS:
                m = CLUSTER_RE.search(p.stem)
                if not m:
                    continue
                cluster_splits[m.group(1)].add(split)

    # 중복 확인
    bad = {cid: sorted(splits) for cid, splits in cluster_splits.items() if len(splits) > 1}
    if not bad:
        print("✅ 모든 클러스터가 한 split에만 존재 (문제 없음)")
    else:
        print(f"❌ {len(bad)}개의 클러스터가 여러 split에 섞여 있음")
        for cid, splits in bad.items():
            print(f"  cluster {cid}: {', '.join(splits)}")

if __name__ == "__main__":
    for f in FOLDERS:
        check_folder(f)


### class 4567도 비율에 맞춰 옮겨보기

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from pathlib import Path
import random, shutil

# ===== 설정 =====
FOLDERS = [
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250721_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250725_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250904_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250930_good_data"),
]
REPORT_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data")

# 비율
RATIOS = {0: 0.70, 1: 0.18, 2: 0.12}
SPLIT_NAME = {0: "train", 1: "val", 2: "test"}
SEED = 13

# 대상: 클래스 4/5/6/7 + BG(라벨 없음/빈 라벨)
TARGET_CLASS_IDS = [4,5,6,7]
EXCLUDE_IF_HAS_012 = True   # ✅ 0/1/2가 하나라도 있으면 대상에서 제외; 012를 무시(포함)하려면 False
HANDLE_BG = True            # ✅ BG도 재분할 대상 포함

# 실제 반영
MATERIALIZE_MODE = "move"   # "move" | "copy" | "symlink"
DRY_RUN = False             # ✅ 실제로 옮기려면 False

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp",
            ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}

def split_dirs(base: Path):
    return {
        0: (base / "images" / "train", base / "labels" / "train"),
        1: (base / "images" / "val",   base / "labels" / "val"),
        2: (base / "images" / "test",  base / "labels" / "test"),
    }

def iter_split_images(base: Path):
    for sid, (dimg, _) in split_dirs(base).items():
        if not dimg.exists(): continue
        for p in dimg.iterdir():
            if p.is_file() and p.suffix in IMG_EXTS:
                yield sid, p

def read_label_lines(txt: Path):
    if not txt.exists(): return []
    try:
        return [ln.strip() for ln in txt.read_text(encoding="utf-8").splitlines() if ln.strip()]
    except:
        return []

def get_class_counts_and_flags(txt: Path):
    lines = read_label_lines(txt)
    is_bg = (len(lines) == 0)
    counts = {k: 0 for k in TARGET_CLASS_IDS}
    has_012 = False
    for s in lines:
        parts = s.split()
        try:
            cid = int(parts[0])
        except:
            continue
        if cid in counts: counts[cid] += 1
        if cid in (0,1,2): has_012 = True
    return counts, is_bg, has_012

def place_file(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    if MATERIALIZE_MODE == "move":
        if src.resolve() == dst.resolve():  # 같은 경로면 스킵
            return
        shutil.move(str(src), str(dst))
    elif MATERIALIZE_MODE == "copy":
        shutil.copy2(str(src), str(dst))
    elif MATERIALIZE_MODE == "symlink":
        try:
            dst.symlink_to(src.resolve())
        except FileExistsError:
            dst.unlink(missing_ok=True)
            dst.symlink_to(src.resolve())
    else:
        raise ValueError("MATERIALIZE_MODE must be move/copy/symlink")

def plan_and_apply(folder: Path):
    random.seed(SEED)

    # 1) 대상 이미지 수집
    items=[]   # (img, lbl, from_split, vec={"imgs":1,"bg":0/1, 4:cnt,5:cnt,6:cnt,7:cnt})
    totals={"imgs":0,"bg":0,**{k:0 for k in TARGET_CLASS_IDS}}

    for sid,img in iter_split_images(folder):
        lbl = folder / "labels" / SPLIT_NAME[sid] / (img.stem + ".txt")
        cnts, is_bg, has_012 = get_class_counts_and_flags(lbl)

        if EXCLUDE_IF_HAS_012 and has_012:
            continue
        has_target = sum(cnts.values()) > 0
        if not has_target and not (HANDLE_BG and is_bg):
            continue

        vec = {"imgs":1, "bg": 1 if (HANDLE_BG and is_bg) else 0}
        vec.update(cnts)
        items.append((img, lbl, sid, vec))

        totals["imgs"] += 1
        totals["bg"]   += vec["bg"]
        for k in TARGET_CLASS_IDS: totals[k] += cnts[k]

    if not items:
        print(f"[INFO] {folder.name}: 대상(4/5/6/7 + BG) 없음")
        return

    # 2) 목표 (이 서브셋 기준)
    targets = {s: RATIOS[s] * totals["imgs"] for s in RATIOS}

    # 3) 단순 비율 기반 배정(이미지 개수 중심) – 필요시 가중 목적함수로 교체 가능
    random.shuffle(items)
    counts = {s:0 for s in RATIOS}
    assignment=[]  # (img,lbl,from_s,to_s,vec)
    for it in items:
        # 현재 목표 대비 가장 부족한 split 선택
        best = min(RATIOS.keys(), key=lambda s: counts[s] - targets[s])
        assignment.append((it[0], it[1], it[2], best, it[3]))
        counts[best] += 1

    # 4) 콘솔 계획 요약
    print(f"\n=== {'DRY-RUN' if DRY_RUN else 'APPLY'}: 4/5/6/7 + BG 재분할 → {folder.name} ===")
    move_preview = 0
    for img,lbl,from_s,to_s,vec in assignment:
        if from_s != to_s:
            print(f"{img.name}: {SPLIT_NAME[from_s]} -> {SPLIT_NAME[to_s]}")
            move_preview += 1
    print(f"대상 이미지: {len(items)} / 이동 예정: {move_preview}")

    # 5) 실제 반영
    if not DRY_RUN:
        for img,lbl,from_s,to_s,vec in assignment:
            if from_s == to_s:
                continue
            img_dst = folder / "images" / SPLIT_NAME[to_s] / img.name
            lbl_dst = folder / "labels" / SPLIT_NAME[to_s] / (img.stem + ".txt")
            place_file(img, img_dst)
            if lbl.exists():     # BG는 라벨 없음 가능
                place_file(lbl, lbl_dst)

    # 6) 최종 분포 집계(이 서브셋에 한정해 리포트)
    final_imgs = {s:0 for s in RATIOS}
    final_bg   = {s:0 for s in RATIOS}
    final_cls  = {s:{k:0 for k in TARGET_CLASS_IDS} for s in RATIOS}
    for _,_,_,to_s,vec in assignment:
        final_imgs[to_s] += 1
        final_bg[to_s]   += vec["bg"]
        for k in TARGET_CLASS_IDS:
            final_cls[to_s][k] += vec[k]

    lines = []
    lines.append(f"[{'DRY' if DRY_RUN else 'OK'}] ({'plan only' if DRY_RUN else 'applied'}) "
                 f"{folder.name} – 4/5/6/7 + BG 재분할  mode={MATERIALIZE_MODE}")
    lines.append(f"  대상 이미지: {len(items)} / 이동: {move_preview}")
    for s in (0,1,2):
        pct = (final_imgs[s]/len(items)*100) if items else 0
        lines.append(f"  {SPLIT_NAME[s]}: {final_imgs[s]} imgs ({pct:.1f}%), BG={final_bg[s]}, "
                     + ", ".join([f"{k}:{final_cls[s][k]}" for k in TARGET_CLASS_IDS]))

    print("\n".join(lines))

    REPORT_DIR.mkdir(parents=True, exist_ok=True)
    out_txt = REPORT_DIR / f"{'dry_' if DRY_RUN else ''}applied_c4567_bg_{folder.name}.txt"
    with out_txt.open("w", encoding="utf-8") as f:
        f.write("\n".join(lines) + "\n")
    print(f"[Saved] {out_txt}")

def run():
    for folder in FOLDERS:
        if not folder.exists():
            print(f"[WARN] 폴더 없음: {folder}")
            continue
        sd = split_dirs(folder)
        if not all(sd[s][0].exists() and sd[s][1].exists() for s in (0,1,2)):
            print(f"[WARN] YOLO 구조(images/labels/<split>) 없음: {folder}")
            continue
        plan_and_apply(folder)

if __name__ == "__main__":
    run()



### 몇개나 섞여있는지 확인 그리고 섞인 곳에 무슨 class가있는지도

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from pathlib import Path
import re
from collections import defaultdict, Counter

# ===== 점검할 데이터 폴더들 =====
FOLDERS = [
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250721_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250725_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250904_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250930_good_data"),
]

# 리포트 저장 위치(각 good_data 상위인 merge_data)
REPORT_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data")

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp",
            ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}
CLUSTER_RE = re.compile(r"_cluster_(\d+)", re.IGNORECASE)

def read_classes(txt_path: Path) -> Counter:
    c = Counter()
    if not txt_path.exists():
        return c
    try:
        for ln in txt_path.read_text(encoding="utf-8").splitlines():
            if not ln.strip():
                continue
            try:
                cid = int(ln.split()[0])
                c[cid] += 1
            except:
                pass
    except:
        pass
    return c

def check_folder(folder: Path):
    lines = []
    lines.append(f"=== Checking {folder.name} ===")
    cluster_info = defaultdict(lambda: {"splits": set(),
                                        "imgs": 0,
                                        "classes": Counter()})
    for split in ("train", "val", "test"):
        img_dir = folder / "images" / split
        if not img_dir.exists():
            continue
        for p in img_dir.rglob("*"):
            if p.is_file() and p.suffix in IMG_EXTS:
                m = CLUSTER_RE.search(p.stem)
                if not m:
                    continue
                cid = m.group(1)
                cluster_info[cid]["splits"].add(split)
                cluster_info[cid]["imgs"] += 1
                lbl = folder / "labels" / split / (p.stem + ".txt")
                cluster_info[cid]["classes"].update(read_classes(lbl))

    bad = {cid: v for cid, v in cluster_info.items() if len(v["splits"]) > 1}
    if not bad:
        msg = "✅ 모든 클러스터가 한 split에만 존재 (문제 없음)"
        print("\n" + msg)
        lines.append(msg)
    else:
        total_imgs = sum(v["imgs"] for v in bad.values())
        msg = (f"❌ {len(bad)}개 클러스터가 여러 split에 섞여 있음 "
               f"(섞인 이미지 총 {total_imgs} 장)")
        print("\n" + msg)
        lines.append(msg + "\n")
        for cid, v in sorted(bad.items(), key=lambda x: int(x[0])):
            splits = ", ".join(sorted(v["splits"]))
            cls_sum = ", ".join(f"{k}:{v['classes'][k]}" for k in sorted(v["classes"]))
            line = (f"  cluster {cid:>4} | splits: {splits:<15} | "
                    f"imgs:{v['imgs']:<3} | classes: {cls_sum or '—'}")
            print(line)
            lines.append(line)

    # txt 저장
    REPORT_DIR.mkdir(parents=True, exist_ok=True)
    out_path = REPORT_DIR / f"cluster_mixed_report_{folder.name}.txt"
    out_path.write_text("\n".join(lines) + "\n", encoding="utf-8")
    print(f"[Saved] {out_path}")

if __name__ == "__main__":
    for f in FOLDERS:
        check_folder(f)



### 데이터 분할 확인

In [ ]:
from pathlib import Path
from collections import Counter, defaultdict

# ====== 설정 ======
ROOT = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data")
TARGETS = [
    ROOT / "20250721_good_data",
    ROOT / "20250725_good_data",
    ROOT / "20250904_good_data",
    ROOT / "20250930_good_data",
]
SPLITS = ["train", "val", "test"]
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp", ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}

# 보고서 파일
out_file = ROOT / "split_class_report.txt"

# ====== 유틸 ======
def read_classes_txt(folder: Path):
    """
    classes.txt가 있으면 그 순서를 그대로 사용.
    없으면 0~7까지 class_i로 가정.
    """
    f = folder / "classes.txt"
    if f.exists():
        names = [ln.strip() for ln in f.read_text(encoding="utf-8").splitlines() if ln.strip()]
        return {i: names[i] for i in range(len(names))}
    return {i: f"class_{i}" for i in range(8)}

def list_images(dirpath: Path):
    return [p for p in dirpath.rglob("*") if p.suffix in IMG_EXTS]

def count_split(folder: Path, split: str, id2name: dict):
    """
    한 데이터셋(folder)의 특정 split(train/val/test)에 대해:
      - per_class: {클래스명: 객체수}
      - bg: BG 이미지 수(라벨 파일 없는 이미지)
      - total_images: 이미지 총 개수
      - labeled_images: 라벨 파일이 있는 이미지 개수
      - total_labels: 객체 총 개수(박스 수)
    """
    img_dir, lbl_dir = folder / "images" / split, folder / "labels" / split
    cls_counter = Counter()
    total_labels = 0

    if lbl_dir.exists():
        for txt in lbl_dir.rglob("*.txt"):
            for line in txt.read_text(encoding="utf-8", errors="ignore").splitlines():
                if not line.strip():
                    continue
                parts = line.split()
                try:
                    cid = int(float(parts[0]))
                except:
                    continue
                cls_counter[cid] += 1
                total_labels += 1

    bg = 0
    stems_lbl = set()
    if img_dir.exists():
        imgs = list_images(img_dir)
        stems_img = {p.stem for p in imgs}
        stems_lbl = {p.stem for p in lbl_dir.rglob("*.txt")} if lbl_dir.exists() else set()
        bg = len(stems_img - stems_lbl)

    total_images = len(list_images(img_dir)) if img_dir.exists() else 0
    labeled_images = len(stems_lbl) if lbl_dir.exists() else 0
    per_class = {id2name.get(k, f"class_{k}"): v for k, v in sorted(cls_counter.items())}
    return per_class, bg, total_images, labeled_images, total_labels

# ====== 총합 집계용 ======
global_total_images = 0
global_total_labeled = 0
global_total_bg = 0
global_total_objects = 0
global_class_totals = Counter()
all_class_names_seen = set()

# 출력 라인(데이터셋별 상세)
report_lines = []

# 선호 출력 순서(있으면 이 순서대로 먼저, 나머지는 이름순)
PREFERRED_ORDER = [
    "Divot", "Fixed_Divot", "Diseased_Grass", "Confused_Object",
    "Pole", "Sprinkler", "Drain", "Golf ball"
]

# ====== 루프: 데이터셋/스플릿 ======
for tgt in TARGETS:
    id2name = read_classes_txt(tgt)
    all_class_names_seen.update(id2name.values())

    report_lines.append(f"\n=== {tgt.name} ===")
    for split in SPLITS:
        per_class, bg, total_images, labeled_images, total_labels = count_split(tgt, split, id2name)

        # 상세 라인
        report_lines.append(
            f"[{split}] images={total_images}, labeled={labeled_images}, BG(no-label)={bg}, total_objects={total_labels}"
        )
        # classes.txt 순서대로 출력(가독성)
        for i in sorted(id2name.keys()):
            cname = id2name[i]
            report_lines.append(f"  {cname:16s}: {per_class.get(cname, 0)}")
        report_lines.append(f"  {'BG':16s}: {bg}")

        # ===== 총합 집계 =====
        global_total_images += total_images
        global_total_labeled += labeled_images
        global_total_bg += bg
        global_total_objects += total_labels
        for cname, cnt in per_class.items():
            global_class_totals[cname] += cnt

# ====== 맨 위에 TOTAL 섹션 만들기 ======
# 클래스 출력 순서 구성
class_order = [c for c in PREFERRED_ORDER if c in all_class_names_seen] + \
              [c for c in sorted(all_class_names_seen) if c not in PREFERRED_ORDER]

total_lines = []
total_lines.append("=== TOTAL (ALL DATASETS) ===")
total_lines.append(
    f"images={global_total_images}, labeled={global_total_labeled}, "
    f"BG(no-label)={global_total_bg}, total_objects={global_total_objects}"
)
for cname in class_order:
    total_lines.append(f"  {cname:16s}: {global_class_totals.get(cname, 0)}")
total_lines.append(f"  {'BG':16s}: {global_total_bg}")

# ====== 파일 저장 (TOTAL을 맨 위로) ======
final_text = "\n".join(total_lines + [""] + report_lines)
out_file.write_text(final_text, encoding="utf-8")

# ====== 콘솔 요약 출력 ======
print(f"✅ 전체 결과를 {out_file} 에 저장했습니다.")
print("\n[TOP SUMMARY]")
print(total_lines[0])
print(total_lines[1])
for line in total_lines[2:]:
    print(line)

print("\n[Per-dataset quick image counts]")
for tgt in TARGETS:
    for split in SPLITS:
        img_count = len(list_images(tgt / "images" / split))
        print(f"{tgt.name} [{split}] images: {img_count}")




### 옛날 증강

In [ ]:
# # augment_balance_train_only_multi.py
# # - 각 good_data 폴더의 train만 증강 (images/train, labels/train)
# # - 출력도 train에 저장(원본 보존)
# # - 중복 억제: 하드캡/쿨다운/다양성 가중치
# # - TARGET은 "train 내 최종 객체 수" 목표(감소 작업은 안 함)

# from pathlib import Path
# from collections import Counter, defaultdict, deque
# import random, csv, os, re
# import cv2
# import albumentations as A

# # ========== 대상 폴더들 ==========
# BASES = [
#     #Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250918_merge_data/20250721_good_data"),
#     #Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250918_merge_data/20250725_good_data"),
#     Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250918_merge_data/20250904_good_data"),
# ]

# # ========== 공통 설정 ==========
# IMG_EXTS = [".jpg", ".jpeg", ".png", ".bmp", ".webp", ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"]
# SEED = 33
# random.seed(SEED)

# # 클래스 매핑(8클래스, 필요시 변경)
# # 0 Divot, 1 Fixed_Divot, 2 Diseased_Grass, 3 Confused_Object, 4 Pole, 5 Sprinkler, 6 Drain, 7 Golf ball
# TARGET = {
#     0: 3500,  # Divot
#     1: 2915,  # Fixed_Divot
#     2: 800,  # Diseased_Grass
#     3: 0,     # Confused_Object
#     4: 150,   # Pole
#     5: 50,    # Sprinkler
#     6: 330,   # Drain
#     7: 200,   # Golf ball
# }
# ALL_CLASS_IDS = list(range(8))
# ALLOW_IDS = set(ALL_CLASS_IDS)

# # --- 중복 억제 파라미터 ---
# MAX_USES_BASE = 1                     # 기본 재사용 상한(낮게)
# MAX_USES_BOOST_PER_CLASS = {          # 부족 클래스일 때 가산치
#     0: 2, 1: 2, 2: 3, 3: 0, 4: 1, 5: 1, 6: 2, 7: 2
# }
# MAX_PER_IMAGE_HARD = 3                # 한 원본에서 절대 최대 생성 수
# RECENT_COOLDOWN = 5                   # 최근 사용 원본 쿨다운 길이

# # 배경(라벨 없는 train 이미지) 증강 배수
# BG_AUG_MULTIPLIER = 2                # 원본 1 + 증강 2 = 총 3배

# # ========== 증강 파이프라인 ==========
# aug = A.Compose([
#     A.HorizontalFlip(p=0.5),
#     A.RandomBrightnessContrast(brightness_limit=(-0.12, 0.09),
#                                contrast_limit=(-0.08, 0.08), p=0.9),
#     A.HueSaturationValue(hue_shift_limit=6, sat_shift_limit=8, val_shift_limit=8, p=0.9),
#     A.RGBShift(r_shift_limit=8, g_shift_limit=8, b_shift_limit=8, p=0.6),
#     A.RandomGamma(gamma_limit=(95, 105), p=0.3),
#     A.OneOf([
#         A.CLAHE(clip_limit=3.0, tile_grid_size=(8, 8), p=0.6),
#         A.ISONoise(color_shift=(0.01, 0.08), intensity=(0.08, 0.35), p=0.4),
#     ], p=0.45),
#     A.OneOf([
#         A.MotionBlur(blur_limit=3, p=0.2),
#         A.GaussianBlur(blur_limit=3, p=0.3),
#         A.Sharpen(alpha=(0.05, 0.2), lightness=(0.9, 1.1), p=0.2),
#     ], p=0.20),
# ], bbox_params=A.BboxParams(
#     format="yolo",
#     label_fields=["class_labels"],
#     min_visibility=0.25
# ))

# # ========== 유틸 ==========
# def read_yolo_label(lbl: Path):
#     boxes, cls = [], []
#     if not lbl.exists(): return boxes, cls
#     with open(lbl, "r") as f:
#         for line in f:
#             p = line.strip().split()
#             if len(p) < 5: continue
#             try: c = int(float(p[0]))
#             except: continue
#             try: x, y, w, h = map(float, p[1:5])
#             except: continue
#             x = min(max(x, 0.0), 1.0); y = min(max(y, 0.0), 1.0)
#             w = min(max(w, 0.0), 1.0); h = min(max(h, 0.0), 1.0)
#             if w <= 0.0 or h <= 0.0: continue
#             boxes.append([x, y, w, h]); cls.append(c)
#     return boxes, cls

# def write_yolo_label(lbl_path: Path, boxes, cls, *, allow_ids=ALLOW_IDS):
#     lbl_path.parent.mkdir(parents=True, exist_ok=True)
#     with open(lbl_path, "w") as f:
#         for c, (x, y, w, h) in zip(cls, boxes):
#             try: c_int = int(float(c))
#             except: continue
#             if allow_ids is not None and c_int not in allow_ids: continue
#             x = float(max(0.0, min(1.0, x)))
#             y = float(max(0.0, min(1.0, y)))
#             w = float(max(0.0, min(1.0, w)))
#             h = float(max(0.0, min(1.0, h)))
#             if w <= 0.0 or h <= 0.0: continue
#             f.write(f"{c_int} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

# def yolo_sanitize(boxes):
#     eps = 1e-6; out = []
#     for x, y, w, h in boxes:
#         l = max(0.0, min(1.0, x - w/2.0))
#         r = max(0.0, min(1.0, x + w/2.0))
#         t = max(0.0, min(1.0, y - h/2.0))
#         b = max(0.0, min(1.0, y + h/2.0))
#         if r - l <= eps or b - t <= eps: continue
#         nx = (l + r)/2.0; ny = (t + b)/2.0; nw = (r - l); nh = (b - t)
#         nx = min(max(nx, eps), 1.0 - eps); ny = min(max(ny, eps), 1.0 - eps)
#         nw = min(max(nw, eps), 1.0 - eps); nh = min(max(nh, eps), 1.0 - eps)
#         out.append([nx, ny, nw, nh])
#     return out

# def collect_background_images(img_dir: Path, lbl_dir: Path):
#     stems_lbl = {p.stem for p in lbl_dir.rglob("*.txt")} if lbl_dir.exists() else set()
#     bg_images = []
#     for ext in IMG_EXTS:
#         for img in img_dir.rglob(f"*{ext}"):
#             if img.stem not in stems_lbl:
#                 bg_images.append(img)
#     return sorted(bg_images)

# def load_image(path: Path):
#     img = cv2.imread(str(path))
#     if img is None:
#         raise RuntimeError(f"Failed to read image: {path}")
#     return img

# def next_unique_name(stem: str, used: set, tag: str):
#     i = 1
#     while True:
#         s = f"{stem}_{tag}{i:04d}"
#         if s not in used:
#             used.add(s)
#             return s
#         i += 1

# def effective_cap_for_file(file_cls_counter: Counter, deficit: Counter):
#     cap = MAX_USES_BASE
#     best_boost = 0
#     for c, n in file_cls_counter.items():
#         if n <= 0: continue
#         if deficit.get(c, 0) > 0:
#             best_boost = max(best_boost, MAX_USES_BOOST_PER_CLASS.get(c, 0))
#     return cap + best_boost

# def current_counts(label_dir: Path):
#     cnt = Counter()
#     for lbl in label_dir.rglob("*.txt"):
#         _, cls = read_yolo_label(lbl)
#         cnt.update(cls)
#     return cnt

# def list_train_stems(img_dir: Path):
#     return {p.stem for p in img_dir.rglob("*") if p.suffix.lower() in IMG_EXTS}

# # ========== 메인 루틴 ==========
# def run_one_base(BASE: Path):
#     IMG_DIR = BASE / "images" / "train"
#     LBL_DIR = BASE / "labels" / "train"
#     IMG_DIR.mkdir(parents=True, exist_ok=True)
#     LBL_DIR.mkdir(parents=True, exist_ok=True)

#     # 유니크명 충돌 방지: 이미 존재하는 stem 모두 확보
#     used_names = set(list_train_stems(IMG_DIR))

#     # 1) 배경 증강
#     bg_list = collect_background_images(IMG_DIR, LBL_DIR)
#     print(f"\n[{BASE.name}] BG images in train: {len(bg_list)}")

#     log_path = BASE / "aug_train_log.csv"
#     with open(log_path, "w", newline="") as f:
#         csv.writer(f).writerow(["src_img","src_lbl","out_img","out_lbl","add_per_class","totals_after"])

#     for bg_img in bg_list:
#         try:
#             img = load_image(bg_img)
#         except Exception:
#             continue
#         for _ in range(BG_AUG_MULTIPLIER):
#             out_stem = next_unique_name(bg_img.stem, used_names, tag="bg")
#             out_img = IMG_DIR / f"{out_stem}{bg_img.suffix.lower()}"
#             out_lbl = LBL_DIR / f"{out_stem}.txt"
#             transformed = aug(image=img, bboxes=[], class_labels=[])
#             cv2.imwrite(str(out_img), transformed["image"])
#             with open(out_lbl, "w") as f:
#                 pass
#     print("  BG augmentation done.")

#     # 2) 라벨 쌍 후보(오직 train)
#     label_files = sorted(LBL_DIR.rglob("*.txt"))
#     candidates = []  # (img_path, lbl_path, file_counts, stem)
#     skipped = 0
#     for lbl in label_files:
#         boxes, cls_list = read_yolo_label(lbl)
#         if not cls_list:
#             continue
#         img = None
#         for ext in IMG_EXTS:
#             cand = IMG_DIR / f"{lbl.stem}{ext}"
#             if cand.exists():
#                 img = cand; break
#         if img is None:
#             skipped += 1
#             continue
#         fcnt = Counter(cls_list)
#         candidates.append((img, lbl, fcnt, lbl.stem))
#     if not candidates:
#         print("  No labeled train pairs. Skip.")
#         return
#     print(f"  Labeled train pairs: {len(candidates)}, skipped(no image match): {skipped}")

#     # 3) 현재(train) 카운트 & 결손
#     cur = current_counts(LBL_DIR)
#     deficit = Counter({c: max(0, TARGET.get(c, 0) - cur.get(c, 0)) for c in ALL_CLASS_IDS})

#     # 4) 반복 증강(중복 억제 포함)
#     use_count = defaultdict(int)
#     recent_imgs = deque(maxlen=RECENT_COOLDOWN)
#     MAX_ITERS = 200000
#     iters = 0

#     def base_score(deficit: Counter, fcnt: Counter):
#         return sum(deficit[c] * fcnt.get(c, 0) for c in ALL_CLASS_IDS if deficit[c] > 0)

#     while any(deficit[c] > 0 for c in ALL_CLASS_IDS) and iters < MAX_ITERS:
#         iters += 1
#         best = None; best_score = 0.0

#         for img, lbl, fcnt, stem in candidates:
#             if img in recent_imgs:
#                 continue
#             if use_count[img] >= MAX_PER_IMAGE_HARD:
#                 continue
#             cap = effective_cap_for_file(fcnt, deficit)
#             if use_count[img] >= cap:
#                 continue

#             sc0 = base_score(deficit, fcnt)
#             if sc0 <= 0:
#                 continue

#             # 다양성 가중치: 많이 쓴 이미지는 불리
#             diversity_w = 1.0 / (1.0 + use_count[img])
#             sc = sc0 * diversity_w

#             if sc > best_score:
#                 best_score = sc
#                 best = (img, lbl, fcnt, stem)

#         if not best:
#             print("  No more useful candidates (after caps/cooldown). Stop.")
#             break

#         img_path, lbl_path, fcnt, stem = best
#         out_stem = next_unique_name(stem, used_names, tag="dup")
#         out_img = IMG_DIR / f"{out_stem}{img_path.suffix.lower()}"
#         out_lbl = LBL_DIR / f"{out_stem}.txt"

#         # 증강
#         img = load_image(img_path)
#         boxes0, cls0 = read_yolo_label(lbl_path)
#         boxes0 = yolo_sanitize(boxes0)

#         transformed = aug(image=img, bboxes=boxes0, class_labels=cls0)
#         aug_img = transformed["image"]
#         aug_boxes = transformed["bboxes"]
#         aug_cls = transformed["class_labels"]

#         keep_boxes, keep_cls = [], []
#         for (x, y, w, h), c in zip(aug_boxes, aug_cls):
#             if w > 0 and h > 0:
#                 keep_boxes.append([float(x), float(y), float(w), float(h)])
#                 keep_cls.append(int(float(c)))

#         # 결손 보강 안 되면 저장 스킵(사용량은 소모)
#         if not keep_cls or sum(Counter(keep_cls)[c] for c in ALL_CLASS_IDS if deficit[c] > 0) <= 0:
#             use_count[img_path] += 1
#             continue

#         # 저장
#         cv2.imwrite(str(out_img), aug_img)
#         write_yolo_label(out_lbl, keep_boxes, keep_cls, allow_ids=ALLOW_IDS)

#         # 카운트/결손 갱신
#         add_cnt = Counter(keep_cls)
#         cur.update(add_cnt)
#         for c in ALL_CLASS_IDS:
#             deficit[c] = max(0, TARGET.get(c, 0) - cur.get(c, 0))

#         use_count[img_path] += 1
#         recent_imgs.append(img_path)

#         # 로그
#         add_str = "{" + ", ".join(f"{k}:{add_cnt.get(k,0)}" for k in ALL_CLASS_IDS) + "}"
#         tot_str = "{" + ", ".join(f"{k}:{cur.get(k,0)}" for k in ALL_CLASS_IDS) + "}"
#         with open(log_path, "a", newline="") as f:
#             csv.writer(f).writerow([str(img_path), str(lbl_path), str(out_img), str(out_lbl), add_str, tot_str])

#         if iters % 50 == 0 or all(deficit[c]==0 for c in ALL_CLASS_IDS):
#             print(f"  [{iters}] cur={dict(cur)} deficit={dict(deficit)} last={out_stem}")

#     print("  Final train counts:", dict(cur))
#     print("  Remaining deficit:", dict(deficit))
#     print(f"  Log: {log_path}")

# # ===== 실행 =====
# if __name__ == "__main__":
#     for base in BASES:
#         run_one_base(base)


### 최신증강

In [ ]:
# augment_balance_train_only_multi.py
# - 각 good_data 폴더의 train만 증강 (images/train, labels/train)
# - 출력도 train에 저장(원본 보존)
# - 중복 억제: 하드캡/쿨다운/다양성 가중치
# - TARGET은 "train 내 최종 객체 수" 목표(감소 작업은 안 함)

from pathlib import Path
from collections import Counter, defaultdict, deque
import random, csv
import cv2
import albumentations as A

# ========== 대상 폴더들 ==========
BASES = [
    #Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250721_good_data"),
    #Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250725_good_data"),
    #Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250904_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/20250930_good_data"),
]

# ========== 공통 설정 ==========
IMG_EXTS = [".jpg", ".jpeg", ".png", ".bmp", ".webp", ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"]
SEED = 13
random.seed(SEED)

# 클래스 매핑(8클래스, 필요시 변경)
# 0 Divot, 1 Fixed_Divot, 2 Diseased_Grass, 3 Confused_Object, 4 Pole, 5 Sprinkler, 6 Drain, 7 Golf ball
TARGET = {
    0: 2400,  # Divot
    1: 3000,  # Fixed_Divot
    2: 100,   # Diseased_Grass
    3: 0,     # Confused_Object
    4: 24,   # Pole
    5: 18,    # Sprinkler
    6: 101,   # Drain
    7: 33,   # Golf ball
}
ALL_CLASS_IDS = list(range(8))
ALLOW_IDS = set(ALL_CLASS_IDS)

# --- 중복 억제 파라미터 ---
MAX_USES_BASE = 1                     # 기본 재사용 상한(낮게)
MAX_USES_BOOST_PER_CLASS = {          # 부족 클래스일 때 가산치
    0: 2, 1: 2, 2: 3, 3: 0, 4: 3, 5: 3, 6: 3, 7: 3
}
MAX_PER_IMAGE_HARD = 1                # 한 원본에서 절대 최대 생성 수
RECENT_COOLDOWN = 5                   # 최근 사용 원본 쿨다운 길이

# 배경(라벨 없는 train 이미지) 증강 배수
BG_AUG_MULTIPLIER = 3                 # 원본 1 + 증강 3 = 총 4배

# ========== 색/감마 변환 정의(플립 제거, 색/감마만) ==========
def _build_color_ops():
    ops = {
        "gamma_bright": A.RandomGamma(gamma_limit=(55, 85), p=1.0),    # 더 밝게
        "gamma_dark":   A.RandomGamma(gamma_limit=(120, 180), p=1.0),  # 더 어둡게
        "warm": A.Compose([
            A.ColorJitter(brightness=0.10, contrast=0.15, saturation=0.08, hue=0.015, p=1.0),
            A.RGBShift(r_shift_limit=(8, 20), g_shift_limit=(-4, 6), b_shift_limit=(-20, -8), p=0.9),
            A.RandomGamma(gamma_limit=(95, 120), p=1.0),
        ], p=1.0),
        "cool": A.Compose([
            A.ColorJitter(brightness=0.08, contrast=0.12, saturation=0.06, hue=0.015, p=1.0),
            A.RGBShift(r_shift_limit=(-20, -8), g_shift_limit=(-6, 6), b_shift_limit=(8, 20), p=0.9),
            A.RandomGamma(gamma_limit=(95, 120), p=1.0),
        ], p=1.0),
        "desat": A.Compose([
            A.HueSaturationValue(hue_shift_limit=0, sat_shift_limit=(-45, -25), val_shift_limit=(-5, 10), p=1.0),
            A.RandomGamma(gamma_limit=(90, 115), p=1.0),
        ], p=1.0),
        "satboost": A.Compose([
            A.HueSaturationValue(hue_shift_limit=0, sat_shift_limit=(20, 45), val_shift_limit=(-5, 10), p=1.0),
            A.RandomGamma(gamma_limit=(90, 115), p=1.0),
        ], p=1.0),
        # 필요시 아주 낮은 확률로 원본 유지:
        # "identity": A.NoOp(p=1.0),
    }
    return ops

COLOR_OPS = _build_color_ops()

# 선택 가중치(원본 유지 비활성화: identity 제외)
WEIGHTS = {
    "gamma_bright": 1.0,
    "gamma_dark":   1.0,
    "warm":         1.0,
    "cool":         1.0,
    "desat":        0.8,
    "satboost":     0.8,
    # "identity":   0.05,  # 정말 원본 유지가 조금 필요하면 낮은 값으로 추가
}

def build_aug(weights: dict) -> A.Compose:
    """좌우반전(0.5) + 색/감마 중 반드시 1개 선택"""
    choices = []
    for name, t in COLOR_OPS.items():
        w = float(weights.get(name, 0.0))
        if w > 0:
            t.p = w  # OneOf의 상대 가중치
            choices.append(t)
    if not choices:
        choices = [A.NoOp(p=1.0)]

    return A.Compose([
        A.HorizontalFlip(p=0.2),     # ← 먼저 플립 시도
        A.OneOf(choices, p=1.0),     # ← 그리고 항상 색/감마 중 1개 적용
    ], bbox_params=A.BboxParams(
        format="yolo",
        label_fields=["class_labels"],
        min_visibility=0.25
    ))

# 최종 aug (여기서 WEIGHTS만 바꾸면 비중이 즉시 반영됨)
aug = build_aug(WEIGHTS)

# ========== 유틸 ==========
def read_yolo_label(lbl: Path):
    boxes, cls = [], []
    if not lbl.exists(): return boxes, cls
    with open(lbl, "r") as f:
        for line in f:
            p = line.strip().split()
            if len(p) < 5: continue
            try:
                c = int(float(p[0]))
            except:
                continue
            try:
                x, y, w, h = map(float, p[1:5])
            except:
                continue
            x = min(max(x, 0.0), 1.0); y = min(max(y, 0.0), 1.0)
            w = min(max(w, 0.0), 1.0); h = min(max(h, 0.0), 1.0)
            if w <= 0.0 or h <= 0.0: continue
            boxes.append([x, y, w, h]); cls.append(c)
    return boxes, cls

def write_yolo_label(lbl_path: Path, boxes, cls, *, allow_ids=ALLOW_IDS):
    lbl_path.parent.mkdir(parents=True, exist_ok=True)
    with open(lbl_path, "w") as f:
        for c, (x, y, w, h) in zip(cls, boxes):
            try:
                c_int = int(float(c))
            except:
                continue
            if allow_ids is not None and c_int not in allow_ids:
                continue
            x = float(max(0.0, min(1.0, x)))
            y = float(max(0.0, min(1.0, y)))
            w = float(max(0.0, min(1.0, w)))
            h = float(max(0.0, min(1.0, h)))
            if w <= 0.0 or h <= 0.0:
                continue
            f.write(f"{c_int} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

def yolo_sanitize(boxes):
    eps = 1e-6; out = []
    for x, y, w, h in boxes:
        l = max(0.0, min(1.0, x - w/2.0))
        r = max(0.0, min(1.0, x + w/2.0))
        t = max(0.0, min(1.0, y - h/2.0))
        b = max(0.0, min(1.0, y + h/2.0))
        if r - l <= eps or b - t <= eps: continue
        nx = (l + r)/2.0; ny = (t + b)/2.0; nw = (r - l); nh = (b - t)
        nx = min(max(nx, eps), 1.0 - eps); ny = min(max(ny, eps), 1.0 - eps)
        nw = min(max(nw, eps), 1.0 - eps); nh = min(max(nh, eps), 1.0 - eps)
        out.append([nx, ny, nw, nh])
    return out

def collect_background_images(img_dir: Path, lbl_dir: Path):
    stems_lbl = {p.stem for p in lbl_dir.rglob("*.txt")} if lbl_dir.exists() else set()
    bg_images = []
    for ext in IMG_EXTS:
        for img in img_dir.rglob(f"*{ext}"):
            if img.stem not in stems_lbl:
                bg_images.append(img)
    return sorted(bg_images)

def load_image(path: Path):
    img = cv2.imread(str(path))
    if img is None:
        raise RuntimeError(f"Failed to read image: {path}")
    return img

def next_unique_name(stem: str, used: set, tag: str):
    i = 1
    while True:
        s = f"{stem}_{tag}{i:04d}"
        if s not in used:
            used.add(s)
            return s
        i += 1

def effective_cap_for_file(file_cls_counter: Counter, deficit: Counter):
    cap = MAX_USES_BASE
    best_boost = 0
    for c, n in file_cls_counter.items():
        if n <= 0: continue
        if deficit.get(c, 0) > 0:
            best_boost = max(best_boost, MAX_USES_BOOST_PER_CLASS.get(c, 0))
    return cap + best_boost

def current_counts(label_dir: Path):
    cnt = Counter()
    for lbl in label_dir.rglob("*.txt"):
        _, cls = read_yolo_label(lbl)
        cnt.update(cls)
    return cnt

def list_train_stems(img_dir: Path):
    return {p.stem for p in img_dir.rglob("*") if p.suffix.lower() in IMG_EXTS}

# ========== 메인 루틴 ==========
def run_one_base(BASE: Path):
    IMG_DIR = BASE / "images" / "train"
    LBL_DIR = BASE / "labels" / "train"
    IMG_DIR.mkdir(parents=True, exist_ok=True)
    LBL_DIR.mkdir(parents=True, exist_ok=True)

    # 유니크명 충돌 방지: 이미 존재하는 stem 모두 확보
    used_names = set(list_train_stems(IMG_DIR))

    # 1) 배경 증강
    bg_list = collect_background_images(IMG_DIR, LBL_DIR)
    print(f"\n[{BASE.name}] BG images in train: {len(bg_list)}")

    log_path = BASE / "aug_train_log.csv"
    with open(log_path, "w", newline="") as f:
        csv.writer(f).writerow(["src_img","src_lbl","out_img","out_lbl","add_per_class","totals_after"])

    for bg_img in bg_list:
        try:
            img = load_image(bg_img)
        except Exception:
            continue
        for _ in range(BG_AUG_MULTIPLIER):
            out_stem = next_unique_name(bg_img.stem, used_names, tag="bg")
            out_img = IMG_DIR / f"{out_stem}{bg_img.suffix.lower()}"
            out_lbl = LBL_DIR / f"{out_stem}.txt"
            transformed = aug(image=img, bboxes=[], class_labels=[])
            cv2.imwrite(str(out_img), transformed["image"])
            with open(out_lbl, "w") as f:
                pass
    print("  BG augmentation done.")

    # 2) 라벨 쌍 후보(오직 train)
    label_files = sorted(LBL_DIR.rglob("*.txt"))
    candidates = []  # (img_path, lbl_path, file_counts, stem)
    skipped = 0
    for lbl in label_files:
        boxes, cls_list = read_yolo_label(lbl)
        if not cls_list:
            continue
        img = None
        for ext in IMG_EXTS:
            cand = IMG_DIR / f"{lbl.stem}{ext}"
            if cand.exists():
                img = cand; break
        if img is None:
            skipped += 1
            continue
        fcnt = Counter(cls_list)
        candidates.append((img, lbl, fcnt, lbl.stem))
    if not candidates:
        print("  No labeled train pairs. Skip.")
        return
    print(f"  Labeled train pairs: {len(candidates)}, skipped(no image match): {skipped}")

    # 3) 현재(train) 카운트 & 결손
    cur = current_counts(LBL_DIR)
    deficit = Counter({c: max(0, TARGET.get(c, 0) - cur.get(c, 0)) for c in ALL_CLASS_IDS})

    # 4) 반복 증강(중복 억제 포함)
    use_count = defaultdict(int)
    recent_imgs = deque(maxlen=RECENT_COOLDOWN)
    MAX_ITERS = 200000
    iters = 0

    def base_score(deficit: Counter, fcnt: Counter):
        return sum(deficit[c] * fcnt.get(c, 0) for c in ALL_CLASS_IDS if deficit[c] > 0)

    while any(deficit[c] > 0 for c in ALL_CLASS_IDS) and iters < MAX_ITERS:
        iters += 1
        best = None; best_score = 0.0

        for img, lbl, fcnt, stem in candidates:
            if img in recent_imgs:
                continue
            if use_count[img] >= MAX_PER_IMAGE_HARD:
                continue
            cap = effective_cap_for_file(fcnt, deficit)
            if use_count[img] >= cap:
                continue

            sc0 = base_score(deficit, fcnt)
            if sc0 <= 0:
                continue

            # 다양성 가중치: 많이 쓴 이미지는 불리
            diversity_w = 1.0 / (1.0 + use_count[img])
            sc = sc0 * diversity_w

            if sc > best_score:
                best_score = sc
                best = (img, lbl, fcnt, stem)

        if not best:
            print("  No more useful candidates (after caps/cooldown). Stop.")
            break

        img_path, lbl_path, fcnt, stem = best
        out_stem = next_unique_name(stem, used_names, tag="dup")
        out_img = IMG_DIR / f"{out_stem}{img_path.suffix.lower()}"
        out_lbl = LBL_DIR / f"{out_stem}.txt"

        # 증강
        img = load_image(img_path)
        boxes0, cls0 = read_yolo_label(lbl_path)
        boxes0 = yolo_sanitize(boxes0)

        transformed = aug(image=img, bboxes=boxes0, class_labels=cls0)
        aug_img = transformed["image"]
        aug_boxes = transformed["bboxes"]
        aug_cls = transformed["class_labels"]

        keep_boxes, keep_cls = [], []
        for (x, y, w, h), c in zip(aug_boxes, aug_cls):
            if w > 0 and h > 0:
                keep_boxes.append([float(x), float(y), float(w), float(h)])
                keep_cls.append(int(float(c)))

        # 결손 보강 안 되면 저장 스킵(사용량은 소모)
        if not keep_cls or sum(Counter(keep_cls)[c] for c in ALL_CLASS_IDS if deficit[c] > 0) <= 0:
            use_count[img_path] += 1
            continue

        # 저장
        cv2.imwrite(str(out_img), aug_img)
        write_yolo_label(out_lbl, keep_boxes, keep_cls, allow_ids=ALLOW_IDS)

        # 카운트/결손 갱신
        add_cnt = Counter(keep_cls)
        cur.update(add_cnt)
        for c in ALL_CLASS_IDS:
            deficit[c] = max(0, TARGET.get(c, 0) - cur.get(c, 0))

        use_count[img_path] += 1
        recent_imgs.append(img_path)

        # 로그
        add_str = "{" + ", ".join(f"{k}:{add_cnt.get(k,0)}" for k in ALL_CLASS_IDS) + "}"
        tot_str = "{" + ", ".join(f"{k}:{cur.get(k,0)}" for k in ALL_CLASS_IDS) + "}"
        with open(log_path, "a", newline="") as f:
            csv.writer(f).writerow([str(img_path), str(lbl_path), str(out_img), str(out_lbl), add_str, tot_str])

        if iters % 50 == 0 or all(deficit[c]==0 for c in ALL_CLASS_IDS):
            print(f"  [{iters}] cur={dict(cur)} deficit={dict(deficit)} last={out_stem}")

    print("  Final train counts:", dict(cur))
    print("  Remaining deficit:", dict(deficit))
    print(f"  Log: {log_path}")

# ===== 실행 =====
if __name__ == "__main__":
    for base in BASES:
        run_one_base(base)



### 데이터 하나로 합치기

In [ ]:
# merge_move_to_top_for_yolo_and_logs.py
from pathlib import Path
import shutil, csv

MERGE_ROOT = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data")
SOURCES = [
    MERGE_ROOT / "20250721_good_data",
    MERGE_ROOT / "20250725_good_data",
    MERGE_ROOT / "20250904_good_data",
    MERGE_ROOT / "20250930_good_data",
]

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp", ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}

def ensure_dirs():
    for base in ["images", "labels"]:
        for split in ["train", "val", "test"]:
            (MERGE_ROOT / base / split).mkdir(parents=True, exist_ok=True)
    (MERGE_ROOT / "logs").mkdir(parents=True, exist_ok=True)
    (MERGE_ROOT / "logs" / "classes_sources").mkdir(parents=True, exist_ok=True)

def map_images(dir_path: Path):
    return {p.stem: p for p in dir_path.glob("*") if p.is_file() and p.suffix in IMG_EXTS}

def move_pair(stem: str, img_path: Path | None, lbl_path: Path | None, split: str, prefix: str):
    new_stem = f"{prefix}__{stem}"
    if img_path and img_path.exists():
        dst_img = MERGE_ROOT / "images" / split / f"{new_stem}{img_path.suffix.lower()}"
        shutil.move(str(img_path), dst_img)
    if lbl_path and lbl_path.exists():
        dst_lbl = MERGE_ROOT / "labels" / split / f"{new_stem}.txt"
        shutil.move(str(lbl_path), dst_lbl)

def move_csvs_and_classes(src_root: Path):
    """소스 폴더 루트에 있는 csv, classes.txt 정리 이동"""
    prefix = src_root.name
    logs_dir = MERGE_ROOT / "logs"

    # CSV들 이동(+ prefix 붙여 충돌 방지)
    for csv_file in src_root.glob("*.csv"):
        dst = logs_dir / f"{prefix}__{csv_file.name}"
        shutil.move(str(csv_file), dst)

    # classes.txt 백업 이동
    cls = src_root / "classes.txt"
    if cls.exists():
        dst = logs_dir / "classes_sources" / f"{prefix}__classes.txt"
        shutil.move(str(cls), dst)

def merge_and_move(src_root: Path):
    prefix = src_root.name
    for split in ["train", "val", "test"]:
        img_dir = src_root / "images" / split
        lbl_dir = src_root / "labels" / split
        if not img_dir.exists() and not lbl_dir.exists():
            continue
        img_map = map_images(img_dir) if img_dir.exists() else {}
        lbl_map = {p.stem: p for p in lbl_dir.glob("*.txt")} if lbl_dir.exists() else {}
        stems = set(img_map) | set(lbl_map)
        for stem in stems:
            move_pair(stem, img_map.get(stem), lbl_map.get(stem), split, prefix)

    # CSV / classes 정리
    move_csvs_and_classes(src_root)

    # 빈 소스 폴더 제거
    shutil.rmtree(src_root, ignore_errors=True)

def read_and_unify_class_names():
    """logs/classes_sources 아래의 classes.txt들을 비교해 통일. 없으면 기본값."""
    defaults = ["Divot","Fixed_Divot","Diseased_Grass","Confused_Object",
                "Pole","Sprinkler","Drain","Golf ball"]
    src_dir = MERGE_ROOT / "logs" / "classes_sources"
    files = sorted(src_dir.glob("*_classes.txt"))
    if not files:
        return defaults, "default"
    # 첫 파일 기준으로 통일성 검사
    def read_names(p: Path):
        return [ln.strip() for ln in p.read_text(encoding="utf-8").splitlines() if ln.strip()]
    base_names = read_names(files[0])
    consistent = True
    for f in files[1:]:
        if read_names(f) != base_names:
            consistent = False
            break
    if not consistent or not base_names:
        # 불일치 시 첫 파일 기준으로 통일하되, 경고 로그 남김
        warn = MERGE_ROOT / "logs" / "classes_mismatch.txt"
        with open(warn, "w", encoding="utf-8") as w:
            w.write("WARNING: classes.txt mismatch among sources. Using first one as canonical.\n")
            for f in files:
                w.write(f"- {f.name}\n")
        return (base_names if base_names else defaults), "mismatch"
    return base_names, "ok"

def write_classes_txt(names: list[str]):
    out = MERGE_ROOT / "classes.txt"
    out.write_text("\n".join(names) + "\n", encoding="utf-8")
    print(f"✔ classes.txt -> {out}")

def write_data_yaml(names: list[str]):
    yaml_path = MERGE_ROOT / "data.yaml"
    with open(yaml_path, "w", encoding="utf-8") as f:
        f.write("train: images/train\n")
        f.write("val: images/val\n")
        f.write("test: images/test\n\n")
        f.write(f"nc: {len(names)}\n")
        f.write("names: [")
        f.write(", ".join(f"\"{n}\"" for n in names))
        f.write("]\n")
    print(f"✔ data.yaml -> {yaml_path}")

def quick_summary():
    for split in ["train", "val", "test"]:
        n_img = len(list((MERGE_ROOT / "images" / split).glob("*")))
        n_lbl = len(list((MERGE_ROOT / "labels" / split).glob("*.txt")))
        print(f"[{split}] images={n_img}, labels={n_lbl}")

def merge_all_aug_logs():
    """logs/ 아래 prefix 붙여 옮긴 CSV 중 aug_* 로그를 하나로 합치고 source 컬럼 추가"""
    logs_dir = MERGE_ROOT / "logs"
    out_path = logs_dir / "merged_aug_log.csv"
    candidates = [p for p in logs_dir.glob("*.csv") if "aug" in p.name.lower()]
    if not candidates:
        return
    header = None
    rows = []
    for p in candidates:
        try:
            with open(p, newline="", encoding="utf-8") as f:
                r = csv.reader(f)
                h = next(r, None)
                if h is None:
                    continue
                if header is None:
                    header = h + ["source_csv"]
                elif h != header[:-1]:
                    # 헤더 다르면 스킵(필요시 매핑 추가 가능)
                    continue
                for row in r:
                    rows.append(row + [p.name])
        except Exception:
            continue
    if header:
        with open(out_path, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            w.writerow(header)
            w.writerows(rows)
        print(f"✔ merged_aug_log.csv -> {out_path}")

if __name__ == "__main__":
    ensure_dirs()
    for src in SOURCES:
        if src.exists():
            print(f"→ Moving from: {src}")
            merge_and_move(src)

    # classes 통합/생성 + data.yaml
    names, status = read_and_unify_class_names()
    write_classes_txt(names)
    write_data_yaml(names)

    # 로그 요약 병합
    merge_all_aug_logs()

    quick_summary()
    print("✅ Done. All files MOVED to top-level. CSV/logs & classes organized.")


### 합쳐진 데이터 로그 출력

In [ ]:
# check_split_distribution.py
from pathlib import Path
from collections import Counter
import csv

MERGE_ROOT = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data")
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp", ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"}
SPLITS = ["train", "val", "test"]

def read_class_names():
    f = MERGE_ROOT / "classes.txt"
    if f.exists():
        names = [ln.strip() for ln in f.read_text(encoding="utf-8").splitlines() if ln.strip()]
        if names: return names
    # fallback
    return ["Divot","Fixed_Divot","Diseased_Grass","Confused_Object","Pole","Sprinkler","Drain","Golf ball"]

CLASS_NAMES = read_class_names()
NC = len(CLASS_NAMES)

def list_images(dirpath: Path):
    return [p for p in dirpath.glob("*") if p.suffix in IMG_EXTS and p.is_file()]

def count_split(split: str):
    img_dir = MERGE_ROOT / "images" / split
    lbl_dir = MERGE_ROOT / "labels" / split
    img_count = 0
    labeled_image_stems = set()
    obj_counter = Counter()
    unknown_counter = 0

    if img_dir.exists():
        img_count = len(list_images(img_dir))
    if lbl_dir.exists():
        for txt in lbl_dir.glob("*.txt"):
            stem = txt.stem
            labeled_image_stems.add(stem)
            for line in txt.read_text(encoding="utf-8", errors="ignore").splitlines():
                parts = line.strip().split()
                if len(parts) < 5: 
                    continue
                try:
                    cid = int(float(parts[0]))
                except:
                    continue
                if 0 <= cid < NC:
                    obj_counter[cid] += 1
                else:
                    unknown_counter += 1

    bg_images = max(0, img_count - len(labeled_image_stems))
    return {
        "images_total": img_count,
        "images_labeled": len(labeled_image_stems),
        "images_bg": bg_images,
        "objs_per_class": obj_counter,
        "objs_unknown": unknown_counter,
    }

def main():
    # 수집
    results = {s: count_split(s) for s in SPLITS}
    # 전체 합산
    total_imgs = sum(results[s]["images_total"] for s in SPLITS)
    total_objs = Counter()
    for s in SPLITS:
        total_objs.update(results[s]["objs_per_class"])
    grand_objs = sum(total_objs.values())

    # 텍스트 리포트 저장
    out_txt = MERGE_ROOT / "split_object_report.txt"
    lines = []
    lines.append(f"Root: {MERGE_ROOT}")
    lines.append(f"Classes({NC}): {', '.join(CLASS_NAMES)}\n")

    for s in SPLITS:
        r = results[s]
        lines.append(f"[{s}] images_total={r['images_total']}, "
                     f"labeled={r['images_labeled']}, bg={r['images_bg']}, "
                     f"objs_total={sum(r['objs_per_class'].values())}, unknown={r['objs_unknown']}")
        # 클래스별
        for i, name in enumerate(CLASS_NAMES):
            lines.append(f"  {i:02d} {name:16s}: {r['objs_per_class'].get(i,0)}")
        # 비율(이 split 내부)
        split_obj_sum = max(1, sum(r["objs_per_class"].values()))
        ratios = ", ".join([f"{name}:{r['objs_per_class'].get(i,0)/split_obj_sum:.3f}" 
                            for i, name in enumerate(CLASS_NAMES)])
        lines.append(f"  per-class ratios: {ratios}\n")

    # 전체 요약
    lines.append("=== OVERALL ===")
    lines.append(f"Total images: {total_imgs}")
    lines.append(f"Total objects: {grand_objs}")
    lines.append("Objects per class (overall):")
    for i, name in enumerate(CLASS_NAMES):
        lines.append(f"  {i:02d} {name:16s}: {total_objs.get(i,0)}")
    if grand_objs > 0:
        ratios_overall = ", ".join([f"{name}:{total_objs.get(i,0)/grand_objs:.3f}" 
                                    for i, name in enumerate(CLASS_NAMES)])
        lines.append(f"Overall ratios: {ratios_overall}")

    out_txt.write_text("\n".join(lines), encoding="utf-8")

    # CSV로도 저장(스플릿별 요약)
    out_csv = MERGE_ROOT / "split_object_report.csv"
    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        header = ["split","images_total","images_labeled","images_bg","objs_total"] + [f"obj_{i}_{CLASS_NAMES[i]}" for i in range(NC)]
        w.writerow(header)
        for s in SPLITS:
            r = results[s]
            row = [s, r["images_total"], r["images_labeled"], r["images_bg"], sum(r["objs_per_class"].values())]
            row += [r["objs_per_class"].get(i,0) for i in range(NC)]
            w.writerow(row)
        # overall 행
        row = ["overall", total_imgs, sum(results[s]["images_labeled"] for s in SPLITS),
               sum(results[s]["images_bg"] for s in SPLITS), grand_objs]
        row += [total_objs.get(i,0) for i in range(NC)]
        w.writerow(row)

    # 콘솔 요약
    print(f"✅ Saved: {out_txt}")
    print(f"✅ Saved: {out_csv}")
    for s in SPLITS:
        r = results[s]
        print(f"[{s}] images={r['images_total']}  labeled={r['images_labeled']}  bg={r['images_bg']}  objs={sum(r['objs_per_class'].values())}")

if __name__ == "__main__":
    main()


### pt를이용한 결과 확인1

In [ ]:
# ==============================
# YOLOv8 Test Metrics Only
# ==============================
from ultralytics import YOLO
from pathlib import Path
import re

# 모델 가중치 & 데이터 설정
WEIGHT    = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251010_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_13_20251010/weights/best.pt"
DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/data.yaml"
model = YOLO(WEIGHT)


# --- test split 평가 ---
results = model.val( 
    data=DATA_YAML, # data.yaml (train/val/test 경로 포함) 
    split="test", # test 세트 사용 
    imgsz= 640, # 입력 해상도 
    batch= 1, # 배치 크기 ## 640 
    rect = True, # default= True false로 하면 이미지 크기 생각없이 무조건 정사각형으로 만듬 
    device=0 # GPU ID (없으면 "cpu") 
)

# # --- Test split 평가 ---
# results = model.val(
#     # ===== 데이터 & 디바이스 =====
#     data=DATA_YAML,    # data.yaml (train/val/test 경로 포함)
#     split="test",      # "val" | "test" | "train" 선택
#     imgsz=640,         # (int) 입력 해상도 (default=640)
#     batch=1,          # (int) 배치 크기 (default=16)
#     rect = True,         # (bool) aspect ratio 유지 여부 (default=True)
#     device=0,          # (int|str|list) 디바이스 (default=None → 자동 선택)

#     # ===== NMS / 임계값 =====
#     conf=0.45,         # (float) confidence threshold (default=0.25)
#     iou=0.5,           # (float) NMS IoU threshold (default=0.6)
#     # max_det=300,       # (int) 이미지당 최대 탐지 개수 (default=300)
#     # agnostic_nms=False,# (bool) 클래스 무관 NMS (default=False)

#     # # ===== 클래스 관련 =====
#     # single_cls=False,  # (bool) 모든 클래스를 하나로 통합 (default=False)
#     # classes=None,      # (list[int]) 특정 클래스만 평가 (default=None → 전체)

#     # # ===== 결과 저장 옵션 =====
#     # save_json=False,   # (bool) COCO json 저장 (default=False)
#     # save_txt=False,    # (bool) txt 저장 (default=False)
#     # save_conf=False,   # (bool) confidence 값도 저장 (default=False)
#     # save_hybrid=False, # (bool) 라벨+예측 병합 저장 (default=False)
#     # plots=False,       # (bool) PR/F1 Curve 등 그래프 저장 (default=False)
#     # verbose=False      # (bool) 상세 로그 출력 (default=False)
# )

print(results)

### pt를 활용한 결과확인2 class별 conf

In [ ]:
# ==============================
# YOLOv8 Test Inference & Metrics (conf / iou 설정 가능 버전)
# ==============================
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
from pathlib import Path
import cv2
import numpy as np

# 1) 모델 불러오기
WEIGHT  = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250918_merge_data_yolov8s_sgd/weights/best.pt"
DATASET = "/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250918_merge_data"
model = YOLO(WEIGHT)

out_dir = Path("runs_yolo/0918_0918_test_sgd")
out_dir.mkdir(parents=True, exist_ok=True)

# (선택) 클래스 이름 매핑: 시각화 라벨에 사용
CLASS_NAMES = {
    0: "Divot",
    1: "Fixed_Divot",
    2: "Diseased_Grass",
    3: "Confused_Object",
    4: "Pole",
    5: "Sprinkler",
    6: "Drain",
    7: "Golf ball",
}
# ⛔ model.names = CLASS_NAMES  # ← 이 줄은 제거 (읽기 전용이라 에러)

# 클래스별 최소 conf 임계치
MIN_CONF_PER_CLASS = {
    0: 0.45,
    1: 0.45,
    2: 0.60,
    3: 0.60,
    4: 0.75,
    5: 0.75,
    6: 0.75,
    7: 0.60,
}

GLOBAL_CONF = 0.001
NMS_IOU = 0.40
MAX_DET = 200

pred_iter = model.predict(
    source=str(Path(DATASET) / "images" / "test"),
    imgsz=1024,
    conf=GLOBAL_CONF,
    iou=NMS_IOU,
    max_det=MAX_DET,
    device=0,
    stream=True,
    save=False,
    verbose=False
)

for res in pred_iter:
    im0 = res.orig_img.copy()
    p = Path(res.path)

    if res.boxes is None or len(res.boxes) == 0:
        cv2.imwrite(str(out_dir / p.name), im0)
        continue

    boxes = res.boxes
    xyxy = boxes.xyxy.cpu().numpy()                 # (N,4)
    conf = boxes.conf.cpu().numpy()                 # (N,)
    cls  = boxes.cls.cpu().numpy().astype(np.int32) # (N,)

    # 클래스별 conf 임계치 적용
    keep = [i for i, c in enumerate(cls) if conf[i] >= MIN_CONF_PER_CLASS.get(int(c), 0.5)]

    annotator = Annotator(im0, line_width=2)
    for i in keep:
        c = int(cls[i])
        lbl = f"{CLASS_NAMES.get(c, str(c))} {conf[i]:.2f}"
        annotator.box_label(xyxy[i], lbl, color=colors(c, True))

    cv2.imwrite(str(out_dir / p.name), annotator.result())


### 학습 sgd

In [2]:
from ultralytics import YOLO
from pathlib import Path
from datetime import datetime
from ultralytics.utils.torch_utils import strip_optimizer
import csv, math
import shutil   # best_map50 저장용

# ==============================
# 실험 관련 기본 설정
# ==============================

# 데이터 설정: train/val 경로, 클래스 수/이름이 정확해야 함
#DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data_only_new_camera/data.yaml"
DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/data.yaml"

# 모델 선택: s(가볍고 빠름) → 결과 좋아지면 m으로 확장
MODEL_NAME = "yolov8s.pt"

# ====== 사용자 지정 하이퍼파라미터 ======
IMG_SIZE  = 1024        # 입력 해상도(정사각). 소물체 ↑. OOM 시 896/768로   
OPTIM     = "SGD"      # Optimizer 선택 ("SGD","AdamW","Adam" 등)
CLS_W     = 0.5        # 클래스 분류 손실 가중(기본≈0.5)
BOX_W     = 7.5       # bbox 회귀 손실 가중(기본≈7.5)|
DFL_W     = 1.5       # Distribution Focal Loss 가중(기본≈1.5)
EPOCHS    = 500          # 필요시 조정
DATA_TAG  = "20251010" # 데이터셋 날짜(데이터셋에 맞게 수정)
RECT_MODE = False      # ✅ rect 설정(True / False) → 폴더명에도 기록됨 디폴트는 False
SEED = 13


# 오늘 날짜 자동 삽입
TODAY     = datetime.now().strftime("%Y%m%d")   # 예: 20250921

# ==============================
# 실행 옵션
# ==============================
SAVE_PERIOD   = 1          # epoch*.pt 저장 주기(1이면 매 에포크 저장)
STRIP_COPIES  = True       # 복사본을 추론용으로 슬림화(optimizer 등 제거)
CLEAN_EPOCHS  = True       # 복사/요약 후 남은 epoch*.pt 삭제(용량 절감)

# ==============================
# 출력 폴더 이름 자동 생성
#    예: 20250919_yolov8s_img896_SGD_cls0.5_box7.5_dfl1.5_rectFalse_20250921
# ==============================
exp_name = (
    f"{DATA_TAG}_data_yolov8s_img{IMG_SIZE}_{OPTIM}"
    f"_cls{CLS_W}_box{BOX_W}_dfl{DFL_W}_rect{str(RECT_MODE)}_seed_{SEED}_{TODAY}"
)
print("실험 폴더 이름:", exp_name)


# ==============================
# 모델 학습 시작
# ==============================
model = YOLO(MODEL_NAME)  # ← 여기만 바뀌면 됩니다

results = model.train(
    # =========================
    # 데이터 & 하드웨어 설정
    # =========================
    data=DATA_YAML,          # (str) 학습/검증/테스트 경로와 클래스 정보를 담은 data.yaml
    imgsz=IMG_SIZE,          # (int) 입력 이미지를 정사각형으로 resize할 크기 (ex: 640)
    batch=-1,                # (int|float) 배치 크기: -1 → GPU 메모리 기준 자동 탐색
    device=0,                # (int|str|list) GPU 인덱스/리스트 또는 'cpu'/'mps'
    workers=8,               # (int) DataLoader 병렬 로드 worker 수 (CPU 코어 상황에 맞춰)

    # =========================
    # 학습 스케줄/최적화
    # =========================
    epochs=EPOCHS,              # (int) 총 학습 epoch 수
    patience=50,             # (int) val metric 개선 없을 때 조기종료 대기 epoch
    lr0=0.01,                # (float) 초기 학습률 (SGD는 0.01 권장, Adam 계열은 더 낮게) 0.01 0.001
    lrf=0.01,                # (float) 최종 학습률 비율 (lr0 * lrf = 마지막 epoch 학습률)
    momentum=0.937,          # (float) SGD 모멘텀 또는 Adam β1 값   
    weight_decay=0.0005,       # (float) L2 정규화 계수 (과적합 방지)   0.01  0.0005

    optimizer=OPTIM,         # (str) 최적화 알고리즘: 'SGD','Adam','AdamW' 등
    cos_lr=False,            # (bool|int) 코사인 학습률 스케줄 사용 여부 (0 → 비활성)
    warmup_epochs=0,         # (float) 워밍업 epoch 수 (0 → 워밍업 없음)
    warmup_momentum=0,       # (float) 워밍업 단계에서 사용할 초기 모멘텀
    warmup_bias_lr=0,        # (float) 워밍업 단계에서 bias 파라미터 학습률

    # =========================
    # 손실 가중치
    # =========================
    box=BOX_W,                 # (float) 바운딩 박스 회귀 손실 가중치
    cls=CLS_W,                 # (float) 클래스 분류 손실 가중치
    dfl=DFL_W,                 # (float) Distribution Focal Loss 가중치 (YOLOv8 box refinement)

    # =========================
    # 데이터 증강(Online Augmentation)
    # =========================
    mosaic=0.0,              # (float) mosaic 증강 비율 (0 → 사용 안함)
    close_mosaic=0,          # (int) 마지막 N epoch 동안 mosaic 비활성 (0 → 비활성화)
    mixup=0.0,               # (float) mixup 비율
    copy_paste=0.0,          # (float) 객체 copy-paste 비율
    hsv_h=0.0,             # (float) Hue 색조 변동 범위
    hsv_s=0.0,               # (float) 채도(Saturation) 변동 범위
    hsv_v=0.0,               # (float) 밝기(Value) 변동 범위
    degrees=0.0,             # (float) 회전 각도 범위
    translate=0.0,           # (float) 이미지 평행이동 비율
    scale=0.0,               # (float) 스케일 변형 비율
    shear=0.0,               # (float) 기울이기 변형 비율
    perspective=0,           # (float) 원근 변형 비율
    erasing=0.0,             # (float) 랜덤 영역 삭제 비율

    # =========================
    # 고급 옵션 및 재현성
    # =========================
    rect=RECT_MODE,          # (bool) 배치 단위로 aspect ratio 유지 여부 (train 기본 False)
    cache="disk",            # (bool|'ram'|'disk') 이미지/라벨 캐시: 'disk'는 I/O 절감
    multi_scale=False,       # (bool) 다중 스케일 학습 활성화 (0 → 비활성)
    single_cls=False,        # (bool) 모든 클래스를 하나로 통합
    classes=None,            # (list[int]|int) 특정 클래스만 학습. 0 → 전체 클래스 사용
    fraction=1.0,            # (float) 데이터셋 사용 비율 (1.0 → 전체 데이터)
    seed=SEED,               # (int) 랜덤 시드 (재현성)
    deterministic=True,      # (bool) 완전 결정론적 알고리즘 사용
    amp=True,                # (bool) 자동 혼합정밀도(fp16) 학습
    freeze=0,                # (int|list) 처음 N개 레이어 혹은 특정 레이어 인덱스 고정
    compile=False,           # (bool|str) torch.compile 사용 (0 → 비활성)
    profile=False,           # (bool) ONNX/TensorRT 프로파일링
    val=True,                # (bool) 학습 중 검증 실행
    save=True,               # (bool) 체크포인트 및 최종 가중치 저장
    save_period=SAVE_PERIOD, # (int) 몇 epoch마다 중간 가중치 저장 (0 → 비활성)

    # =========================
    # 결과 저장
    # =========================
    project="/home/dw/ws_job_msislab/Golf_Project/runs_yolo",  # (str) 상위 프로젝트 폴더
    name=exp_name,           # (str) 프로젝트 하위 폴더명
    exist_ok=True            # (bool) 동일 이름 폴더가 있어도 덮어쓰기 허용
)


# 🔹 sum4 가중치 (사용자가 직접 수정 가능)
WEIGHTS_SUM4 = {
    "precision": 0.20,
    "recall":    0.30,
    "map50":     0.25,
    "map5095":   0.25,
}


# ==============================
# 후처리: results.csv 기반 베스트 체크포인트 정리
# ==============================
proj_dir    = Path("/home/dw/ws_job_msislab/Golf_Project/runs_yolo")
exp_dir     = proj_dir / exp_name
weights_dir = exp_dir / "weights"
csv_path    = exp_dir / "results.csv"

if not csv_path.exists():
    raise FileNotFoundError(f"results.csv을 찾을 수 없습니다: {csv_path}")

CANDIDATES = {
    "precision": ["metrics/precision(B)", "metrics/precision", "precision"],
    "recall":    ["metrics/recall(B)",    "metrics/recall",    "recall"],
    "map50":     ["metrics/mAP50(B)",     "metrics/mAP50",     "map50"],
    "map5095":   ["metrics/mAP50-95(B)",  "metrics/mAP50-95",  "mAP50-95"],
}

def read_results_csv(path: Path):
    with path.open("r", newline="") as f:
        reader = csv.DictReader(f)
        rows = list(reader)
        fields = reader.fieldnames or []
    return rows, fields

def pick_col(cands, fieldnames):
    for c in cands:
        if c in fieldnames: return c
    raise KeyError(f"CSV에 해당 메트릭 컬럼이 없음: {cands}")

def to_float_safe(v):
    try:
        x = float(v); return x if not math.isnan(x) else -1.0
    except: return -1.0

rows, fields = read_results_csv(csv_path)
col_prec    = pick_col(CANDIDATES["precision"], fields)
col_rec     = pick_col(CANDIDATES["recall"],    fields)
col_map50   = pick_col(CANDIDATES["map50"],     fields)
col_map5095 = pick_col(CANDIDATES["map5095"],   fields)

# ==============================
# Top3 관리 함수
# ==============================
def update_top3(top3, value, epoch):
    b_val, b_ep = top3[0]; s_val, s_ep = top3[1]; t_val, t_ep = top3[2]
    if value > b_val or (value == b_val and (b_ep == -1 or epoch < b_ep)):
        if b_ep != -1 and b_ep != epoch:
            t_val, t_ep = s_val, s_ep
            s_val, s_ep = b_val, b_ep
        b_val, b_ep = value, epoch
    elif epoch != b_ep and (value > s_val or (value == s_val and (s_ep == -1 or epoch < s_ep))):
        if s_ep != -1 and s_ep != epoch:
            t_val, t_ep = s_val, s_ep
        s_val, s_ep = value, epoch
    elif epoch not in (b_ep, s_ep) and (value > t_val or (value == t_val and (t_ep == -1 or epoch < t_ep))):
        t_val, t_ep = value, epoch
    top3[0], top3[1], top3[2] = (b_val, b_ep), (s_val, s_ep), (t_val, t_ep)

top3 = {m:[(-1.0,-1)]*3 for m in ["precision","recall","map50","map5095","sum4"]}

for r in rows:
    if not r.get("epoch"): continue
    e_csv = int(r["epoch"])
    p,rc,m50,m95 = map(to_float_safe, [r.get(col_prec), r.get(col_rec), r.get(col_map50), r.get(col_map5095)])
    update_top3(top3["precision"], p, e_csv)
    update_top3(top3["recall"],    rc, e_csv)
    update_top3(top3["map50"],     m50, e_csv)
    update_top3(top3["map5095"],   m95, e_csv)
    sum4 = (p*WEIGHTS_SUM4["precision"] +
            rc*WEIGHTS_SUM4["recall"] +
            m50*WEIGHTS_SUM4["map50"] +
            m95*WEIGHTS_SUM4["map5095"])
    update_top3(top3["sum4"], sum4, e_csv)

# ==============================
# 체크포인트 복사
# ==============================
best_file, last_file = weights_dir/"best.pt", weights_dir/"last.pt"
kept_files = set([f.name for f in (best_file,last_file) if f.exists()])
summary_lines=[]

def ckpt_path_from_csv_epoch(e_csv: int) -> Path:
    return weights_dir / f"epoch{e_csv-1}.pt"

def same_file(a: Path, b: Path) -> bool:
    return a.exists() and b.exists() and a.read_bytes() == b.read_bytes()

def maybe_copy(tag_prefix, rank, epoch_csv, value):
    if epoch_csv < 0 or value < 0: return
    src = ckpt_path_from_csv_epoch(epoch_csv)
    if not src.exists(): return
    if same_file(src,best_file) or same_file(src,last_file): return
    dst = weights_dir / f"{tag_prefix}_{rank}_epoch{epoch_csv}_{value:.5f}.pt"
    shutil.copy2(src,dst)
    if STRIP_COPIES: strip_optimizer(str(dst))
    kept_files.add(dst.name)
    summary_lines.append(f"{tag_prefix} {rank}: epoch={epoch_csv}, value={value:.6f} -> {dst.name}")

for metric in ["precision","recall","map50","map5095","sum4"]:
    for idx,rank in enumerate(["BEST","SECOND","THIRD"]):
        v,e = top3[metric][idx]
        maybe_copy(metric,rank,e,v)

# ==============================
# 남은 epoch 삭제
# ==============================
if CLEAN_EPOCHS:
    for p in weights_dir.glob("epoch*.pt"):
        if p.name not in kept_files:
            p.unlink(missing_ok=True)

summary_path = exp_dir / "best_by_metric_summary.txt"
with summary_path.open("w") as f:
    f.write("=== Top3 by Metric (with weighted sum4) ===\n")
    f.write(f"WEIGHTS_SUM4 = {WEIGHTS_SUM4}\n\n")
    for line in summary_lines: f.write(line+"\n")

print("\n[완료] 보존된 체크포인트:")
for n in sorted(kept_files): print(" -",n)
print(f"요약 저장됨: {summary_path}")


실험 폴더 이름: 20251010_data_yolov8s_img1024_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_13_20251013
New https://pypi.org/project/ultralytics/8.3.213 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.202 🚀 Python-3.10.12 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=0, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/dw/ws_job_msislab/Golf_Project/data/for_study/20251010_merge_data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=500, erasing=0.0, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mas

KeyboardInterrupt: 